# Coding Preparation

## Import the Necessary Python Package

In [1]:
import os
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.ticker as ticker

import pandas as pd
from trino.dbapi import connect 
from sqlalchemy import create_engine
import time

import geopandas as gpd

class TrinoEngine():
    def __init__(self):
        conn = connect(
            host="localhost",
            port=9090,
            catalog="cuebiq"
        )
        self.cur = conn.cursor()
        self.engine = create_engine("trino://localhost:9090/cuebiq/")
    
    def execute_statement(self, query:str) -> list:
        """
        Create and drop statements.
        """
        self.cur.execute(query)
        return self.cur.fetchall()
    
    def read_sql(self, query:str) -> pd.DataFrame: 
        """
        Select and insert into operations.
        """
        return pd.read_sql(query, self.engine)

sql_engine = TrinoEngine()

# def read_sql(query:str) -> pd.DataFrame:
#     return pd.read_sql(query, connection)

In [2]:
# !pip install pydeck -q -q

In [3]:
pd.options.mode.chained_assignment = None

In [4]:
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")

## Create SQL Enghine Connect

In [5]:
schema_name = {'cda': 'cuebiq.paas_cda_pe_v2'}

stop_table = f"{schema_name['cda']}.stop_uplevelled"
geography_table = f"{schema_name['cda']}.geography_registry"
device_daily_metrics_table =f"{schema_name['cda']}.device_metrics"

In [6]:
%reload_ext sql
%config SqlMagic.autocommit=False
%config SqlMagic.autolimit=0
%config SqlMagic.displaylimit=200
%sql trino://localhost:9090/cuebiq/

'Connected: @cuebiq/'

In [7]:
# %sql trino://localhost:9090/cuebiq/

In [8]:
exposure_scaled = "dedicated.exposure_scaled"

In [9]:
from sqlalchemy.engine import create_engine
con = create_engine(f"trino://localhost:9090/dedicated/exposure_scaled")
con_inc = create_engine(f"trino://localhost:9090/dedicated/incrementors")

In [10]:
# %sql create schema if not exists dedicated.exposure_scaled

In [11]:
# %sql create schema if not exists dedicated.incrementors

In [12]:
# to delete curr data
# %sql delete from dedicated.exposure_scaled.county_urban_prod
# %sql delete from dedicated.exposure_scaled.county_rural_prod
# %sql delete from dedicated.incrementors.failed_counties_rural
# %sql delete from dedicated.incrementors.failed_counties_urban

In [13]:
# create a table called dedicated.exposure_scaled.county_rural_prod and county_urban_prod which has the following columns and types:
# - block_group_id: varchar
# - white_weighted_exposure: real
# - black_weighted_exposure: real
# - asian_weighted_exposure: real
# - hispanic_weighted_exposure: real
# %sql create table if not exists dedicated.exposure_scaled.county_rural_prod (tract_id varchar, white_weighted_exposure varchar, black_weighted_exposure varchar, asian_weighted_exposure varchar, hispanic_weighted_exposure varchar, other_weighted_exposure varchar, total_weighted_exposure varchar, date varchar, count_people varchar) with (format='CSV')
# %sql create table if not exists dedicated.exposure_scaled.county_urban_prod (tract_id varchar, white_weighted_exposure varchar, black_weighted_exposure varchar, asian_weighted_exposure varchar, hispanic_weighted_exposure varchar, other_weighted_exposure varchar, total_weighted_exposure varchar, date varchar, count_people varchar) with (format='CSV')

# %sql create table if not exists dedicated.exposure_scaled.county_rural_prod_2 (tract_id varchar, white_weighted_exposure varchar, black_weighted_exposure varchar, asian_weighted_exposure varchar, hispanic_weighted_exposure varchar, other_weighted_exposure varchar, total_weighted_exposure varchar, date varchar, count_people varchar) with (format='CSV')
# %sql create table if not exists dedicated.exposure_scaled.county_urban_prod_2 (tract_id varchar, white_weighted_exposure varchar, black_weighted_exposure varchar, asian_weighted_exposure varchar, hispanic_weighted_exposure varchar, other_weighted_exposure varchar, total_weighted_exposure varchar, date varchar, count_people varchar) with (format='CSV')

# %sql create table if not exists dedicated.exposure_scaled.county_rural_prod_3 (tract_id varchar, white_weighted_exposure varchar, black_weighted_exposure varchar, asian_weighted_exposure varchar, hispanic_weighted_exposure varchar, other_weighted_exposure varchar, total_weighted_exposure varchar, date varchar, count_people varchar) with (format='CSV')
# %sql create table if not exists dedicated.exposure_scaled.county_urban_prod_3 (tract_id varchar, white_weighted_exposure varchar, black_weighted_exposure varchar, asian_weighted_exposure varchar, hispanic_weighted_exposure varchar, other_weighted_exposure varchar, total_weighted_exposure varchar, date varchar, count_people varchar) with (format='CSV')

In [14]:
# %sql create table if not exists dedicated.incrementors.dates (date varchar)
# %sql create table if not exists dedicated.incrementors.counties (county varchar)
# %sql create table if not exists dedicated.incrementors.current_rural (county varchar, date varchar)
# %sql create table if not exists dedicated.incrementors.current_urban (county varchar, date varchar)
# %sql create table if not exists dedicated.incrementors.failed_counties_rural (county varchar, date varchar)
# %sql create table if not exists dedicated.incrementors.failed_counties_urban (county varchar, date varchar)

In [15]:
# %sql show tables from dedicated.exposure_scaled

## Define Fuction for Inserting Data into SQL Tables

In [16]:
def insert_into_db(tbl_name, df, interval):
    '''
    helper function to insert a dataframe into a sql table.
    
    tbl_name: string
    df: pd df
    '''
    
    num_rows = df.shape[0]
    range_start = 0
    range_end = interval
    
    while range_start < num_rows:
        if range_end > num_rows:
            range_end = num_rows
        current_df = df.iloc[range_start:range_end]
        current_df.to_sql(
            tbl_name, 
            con, 
            index=False, 
            if_exists="append", 
            method="multi")
        print('success: now at row '+str(range_end))
        range_start += interval
        range_end += interval

def insert_into_db_inc(tbl_name, df, interval):
    '''
    helper function to insert a dataframe into a sql table.
    
    tbl_name: string
    df: pd df
    '''
    
    num_rows = df.shape[0]
    range_start = 0
    range_end = interval
    
    while range_start < num_rows:
        if range_end > num_rows:
            range_end = num_rows
        current_df = df.iloc[range_start:range_end]
        current_df.to_sql(
            tbl_name, 
            con_inc, 
            index=False, 
            if_exists="append", 
            method="multi")
        print('success: now at row '+str(range_end))
        range_start += interval
        range_end += interval

In [17]:
# counties = pd.read_html('https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county',)[1]
# state_abr = pd.read_html('https://simple.wikipedia.org/wiki/List_of_U.S._states_by_traditional_abbreviation',)[0]
# counties['FIPS'] = counties['FIPS'].astype(str).str.zfill(5)
# counties['State or equivalent'] = counties['State or equivalent'].replace({'Hawaiʻi' : 'Hawaii'})
# state_abr = state_abr[['State','Otherabbreviations']].rename(columns={'Otherabbreviations':'abr'})
# counties = counties.merge(state_abr, left_on='State or equivalent', right_on='State')[['FIPS', 'abr']]
# counties['admin2'] = 'US.' + counties['abr'] + '.' + counties['FIPS'].str[2:]

# insertion
# insert_into_db_inc('counties', pd.DataFrame({'county':counties['admin2'].values}),500)

# Input Crosswalk File and merge Demographic Data

## Read the "Weights" dataframe

The home block groups are provided by Cuebiq are in 2010 administrative boundaries. Given that the 2017-2021 American Community Survey (ACS) most closely represents the 2022 population, we use Relationship Files provided by the U.S. Census Bureau to crosswalk the ACS data, which uses 2020 boundaries, to 2010 boundaries. Using the Relationship FIles, which provides separate “parts” of all 2020 tracts, their corresponding 2010 tract, and the area of the 2010 tract it represents. Each tract “part” is given a weight:  
$$W_p=\frac{\text { area }_{p, 2020}}{\text { area }_{p, 2010}}$$
Where for each “part” , ${area}_{p, 2020}$ is the area that this part takes up in a 2020 Census tract and ${area}_{p, 2010}$ is the area that this part takes up in a 2010 Census tract.

In [18]:
# weights = pd.read_csv('../External_data/census_2020_2010cw_weights_tract.csv',dtype={'home_tract_id':str})
# weights

In [19]:
# sql_engine.read_sql(
# f"""
# drop table dedicated.exposure_scaled.weights
# """)

In [20]:
# %sql create table if not exists dedicated.exposure_scaled.weights (weights real, home_tract_id varchar)
# insert_into_db("weights", weights[['weights','home_tract_id']],2000)

In [21]:
weights = sql_engine.read_sql(f"""
          select *
          from dedicated.exposure_scaled.weights
          """)

weights.info()
weights.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73509 entries, 0 to 73508
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   weights        73509 non-null  float64
 1   home_tract_id  73509 non-null  object 
dtypes: float64(1), object(1)
memory usage: 1.1+ MB


,weights,home_tract_id
0,3.481295,29047021309
1,3.783115,48375015200
2,8.083858,36003950900
3,4.488793,29121960300
4,4.332431,29121960500


## Read the "Demographic" data and Merge the "Weight" Dataframe into together

In [22]:
cw_2010_2020 = sql_engine.read_sql(f"""
          select *
          from dedicated.exposure_scaled.cw_2010_2020_tract
          """)

In [23]:
cw_2010_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73669 entries, 0 to 73668
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gisjoin           73669 non-null  object 
 1   hispanic          73669 non-null  float64
 2   total_pop         73669 non-null  float64
 3   white             73669 non-null  float64
 4   black             73669 non-null  float64
 5   indigenous        73669 non-null  float64
 6   asian             73669 non-null  float64
 7   pac_isl           73669 non-null  float64
 8   other             73669 non-null  float64
 9   two_more          73669 non-null  float64
 10  ba                73669 non-null  float64
 11  ma                73669 non-null  float64
 12  prof              73669 non-null  float64
 13  phd               73669 non-null  float64
 14  median_income     71759 non-null  float64
 15  total_school_pop  73666 non-null  float64
dtypes: float64(15), object(1)
memory usage: 

In [24]:
# create the new columns by extracting from the 'gisjoin' column.
cw_2010_2020['state'] = cw_2010_2020['gisjoin'].astype(str).str[1:3]
cw_2010_2020['county'] = cw_2010_2020['gisjoin'].astype(str).str[4:7]
cw_2010_2020['tract'] = cw_2010_2020['gisjoin'].astype(str).str[-6:]

cw_2010_2020['home_tract_id'] = cw_2010_2020['state']+cw_2010_2020['county']+cw_2010_2020['tract']

# merge 'weight' and 'cw_2010_2020' into together
cw_2010_2020 = cw_2010_2020.merge(weights[['weights','home_tract_id']],left_on='home_tract_id',right_on='home_tract_id')
cw_2010_2020['other'] = cw_2010_2020['other']+cw_2010_2020['indigenous'] +cw_2010_2020['pac_isl'] + cw_2010_2020['two_more'] 

<font color='red' size=5>Question 1: it looks like we should operate this code to transfer 2020 boundary into 2010 boundary, but we didn't do anything in original data</font>

In [25]:
# cw_2010_2020.loc[:,['hispanic', 'total_pop', 'white', 'black',
#        'indigenous', 'asian', 'pac_isl', 'other', 'two_more', 'ba', 'ma',
#        'prof', 'phd', 'total_school_pop']] = cw_2010_2020[['hispanic', 'total_pop', 'white', 'black',
#        'indigenous', 'asian', 'pac_isl', 'other', 'two_more', 'ba', 'ma',
#        'prof', 'phd', 'total_school_pop']].multiply(cw_2010_2020["weights"], axis="index")

In [26]:
# filited the outlier of the 'cw_2010_2020' dataframe

cw_2010_2020 = cw_2010_2020[(cw_2010_2020['hispanic']>=1)|\
                            (cw_2010_2020['total_pop']>=1)|\
                            (cw_2010_2020['white']>=1)|\
                            (cw_2010_2020['black']>=1)|\
                            (cw_2010_2020['asian']>=1)|\
                            (cw_2010_2020['other']>=1)]

cw_2010_2020.info()
cw_2010_2020.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73323 entries, 0 to 73508
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gisjoin           73323 non-null  object 
 1   hispanic          73323 non-null  float64
 2   total_pop         73323 non-null  float64
 3   white             73323 non-null  float64
 4   black             73323 non-null  float64
 5   indigenous        73323 non-null  float64
 6   asian             73323 non-null  float64
 7   pac_isl           73323 non-null  float64
 8   other             73323 non-null  float64
 9   two_more          73323 non-null  float64
 10  ba                73323 non-null  float64
 11  ma                73323 non-null  float64
 12  prof              73323 non-null  float64
 13  phd               73323 non-null  float64
 14  median_income     71708 non-null  float64
 15  total_school_pop  73320 non-null  float64
 16  state             73323 non-null  object

,gisjoin,hispanic,total_pop,white,black,indigenous,asian,pac_isl,other,two_more,ba,ma,prof,phd,median_income,total_school_pop,state,county,tract,home_tract_id,weights
0,G1200630211000,129.00000,4727.0000,3676.00000,673.00000,0.000000,13.000000,0.0,236.000000,236.000000,257.000000,141.00000,0.000000,12.000000,41957.000,472.00000,12,063,211000,12063211000,3.065499
1,G0600830002402,5797.66060,7235.3286,895.43823,94.01805,0.000000,326.837160,0.0,121.374466,121.374466,358.583980,65.35339,3.556893,12.893738,70760.000,597.09875,06,083,002402,06083002402,5.979610
2,G1200630210900,203.61212,2851.0771,1895.05760,646.57880,13.532524,21.054125,0.0,84.774534,71.242010,115.468605,82.57761,0.000000,14.591327,43267.734,306.43936,12,063,210900,12063210900,2.845386
3,G1200630210500,95.00298,3767.0166,2610.01120,1039.99930,0.000705,0.000686,0.0,22.002783,13.001150,223.004750,146.00380,36.999980,45.000187,51224.070,437.99802,12,063,210500,12063210500,5.083693
4,G0600830002102,1418.00000,2240.0000,704.00000,15.00000,8.000000,37.000000,0.0,66.000000,58.000000,237.000020,80.00000,10.000000,4.000000,74191.000,132.00000,06,083,002102,06083002102,14.177216


# Create Functions

## 1.The function of `'get_stop_data'`   

**Explanation:**  
This function is used to retrieve information from the database based on the given county and date. It performs a series of SQL query operations, selecting data from two different tables(<font color='blue'> <u>**"stop_table"**</u> and <u>**"device_daily_metrics_table"**</u></font>) and associating them together through join operations. The final query result is returned as a dataframe.  
- `get_stop_data`: input one county at given date  
- `get_stop_data_many`: input multiple counties at give date

In [27]:
# Block group

def get_stop_data(county, date):
    date_str = date.strftime('%Y%m%d')
    stop_data = sql_engine.read_sql(
        f"""
        select a.*,b.max_distance_from_home_meters,b.local_date,
           b.total_home_dwell_time_minutes,
           b.home_known_flag, b.work_known_flag,
           b.home_admin1_id, b.home_admin2_id, b.home_block_group_id,
           b.work_admin1_id, b.work_admin2_id from 
        (select {stop_table}.*,
        cast(from_iso8601_timestamp(stop_zoned_datetime) as date) as stop_date, cast(hour(from_iso8601_timestamp(stop_zoned_datetime)) as int) as stop_hour
        from {stop_table}
           where processing_date between {int(date_str)} and {int(date_str) + 2}
           and admin2_id='{county}' and dwell_time_minutes <=720
           and cast(year(from_iso8601_timestamp(stop_zoned_datetime))as int)={date.year}
           and cast(month(from_iso8601_timestamp(stop_zoned_datetime)) as int)={date.month}
           and cast(day(from_iso8601_timestamp(stop_zoned_datetime)) as int)={date.day}) as a
           join
           (select cuebiq_id, max_distance_from_home_meters,local_date,
           max_distance_from_work_meters, total_dwell_time_minutes,
           total_home_dwell_time_minutes, total_work_dwell_time_minutes,
           home_known_flag, work_known_flag, home_country_code,
           home_admin1_id, home_admin2_id, home_block_group_id,
           home_tag_update_date, work_country_code, work_admin1_id,
           work_admin2_id, work_block_group_id, work_tag_update_date
              from {device_daily_metrics_table}
            where local_date = {int(date_str)} AND
             home_known_flag = True) as b
             on a.cuebiq_id = b.cuebiq_id
        """
    )
    return stop_data

# Block group

def get_stop_data_many(counties, date):
    date_str = date.strftime('%Y%m%d')
    placeholders= ', '.join("'"+unused+"'" for unused in counties)
    query= '(%s)' % placeholders
    stop_data = sql_engine.read_sql(
        f"""
        select a.*,b.max_distance_from_home_meters,b.local_date,
           b.total_home_dwell_time_minutes,
           b.home_known_flag, b.work_known_flag,
           b.home_admin1_id, b.home_admin2_id, b.home_block_group_id,
           b.work_admin1_id, b.work_admin2_id from 
        (select {stop_table}.*,
        cast(from_iso8601_timestamp(stop_zoned_datetime) as date) as stop_date, cast(hour(from_iso8601_timestamp(stop_zoned_datetime)) as int) as stop_hour
        from {stop_table}
           where processing_date between {int(date_str)} and {int(date_str) + 2}
           and admin2_id in {query}
           and dwell_time_minutes <=720
           and cast(year(from_iso8601_timestamp(stop_zoned_datetime))as int)={date.year}
           and cast(month(from_iso8601_timestamp(stop_zoned_datetime)) as int)={date.month}
           and cast(day(from_iso8601_timestamp(stop_zoned_datetime)) as int)={date.day}) as a
           join
           (select cuebiq_id, max_distance_from_home_meters,local_date,
           max_distance_from_work_meters, total_dwell_time_minutes,
           total_home_dwell_time_minutes, total_work_dwell_time_minutes,
           home_known_flag, work_known_flag, home_country_code,
           home_admin1_id, home_admin2_id, home_block_group_id,
           home_tag_update_date, work_country_code, work_admin1_id,
           work_admin2_id, work_block_group_id, work_tag_update_date
              from {device_daily_metrics_table}
            where local_date = {int(date_str)} AND
             home_known_flag = True) as b
             on a.cuebiq_id = b.cuebiq_id
        """
    )
    return stop_data

<font color='blue'>**For Example:  
function <u>"1:get_stop_data"</u>**</font>

In [ ]:
# Defining input parameters: county and date
county = 'US.IL.031'
date_unformatted = '2022-08-01'

# to record the time when the program starts executing
exposure_start_time = time.time()

date = datetime.fromisoformat(date_unformatted) 
stop_data_test = get_stop_data(county, date)  
stop_data_test['geohash_id'] =  stop_data_test['geohash_id'].str[:8] # transfer into geohash_8 ids

# to record the time when the program finish executing
print("total executing time", (time.time() - exposure_start_time)/60, "minutes to run")

stop_data_test.info()
stop_data_test.head()

## 2.The function of `'remove_nonhuman_devices'`   

**Explanation:**  
This function is used to remove "nonhuman_devices" records, retain only those records with fewer than "20" time on the same "geohash_id". This can be used to filter out those records which may be the result of stationary equipment rather than human activity.

In [29]:
def remove_nonhuman_devices(stop_data):
    """
    Count the number of stays that have the same geohash id, latitude, and longitude. 
    If that number is GREATER THAN one, then filter out that data. 
    """
    
    print("size of data before filtering: ", len(stop_data))
    
    grouped_data = stop_data.groupby(['geohash_id', 'lat', 'lng'])['lat'].count().reset_index(name="count")
    ## Let's use an upper limit of 20 to account for people who may be staying in one place for a while.
    filtered_data = grouped_data[grouped_data['count'] <20]
    geohashs_to_keep = list(filtered_data['geohash_id'])
    data = stop_data[stop_data['geohash_id'].isin(geohashs_to_keep)]
    
    print("size of data after filtering: ", len(data))
    
    return data

<font color='blue'>**For Example:   
function <u>"2:remove_nonhuman_devices"</u>**</font>

In [30]:
# to record the time when the program starts executing
exposure_start_time = time.time()

filtered_stop_data_test = remove_nonhuman_devices(stop_data_test)

# to record the time when the program finish executing
print("total executing time", (time.time() - exposure_start_time)/60, "minutes to run")

filtered_stop_data_test.info()
filtered_stop_data_test.head()

size of data before filtering:  580401
size of data after filtering:  497203
total executing time 0.03366692860921224 minutes to run
<class 'pandas.core.frame.DataFrame'>
Int64Index: 497203 entries, 0 to 580400
Data columns (total 31 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   cuebiq_id                      497203 non-null  int64  
 1   device_type_code               497203 non-null  object 
 2   lat                            497203 non-null  float64
 3   lng                            497203 non-null  float64
 4   geohash_id                     497203 non-null  object 
 5   cluster_size                   497203 non-null  int64  
 6   avg_distance_meters            497203 non-null  float64
 7   avg_accuracy_meters            497199 non-null  float64
 8   std_accuracy_meters            497199 non-null  float64
 9   dwell_time_minutes             497203 non-null  float64
 10  stop_zoned_datetime

,cuebiq_id,device_type_code,lat,lng,geohash_id,cluster_size,avg_distance_meters,avg_accuracy_meters,std_accuracy_meters,dwell_time_minutes,stop_zoned_datetime,timezone_offset_seconds,admin1_id,admin2_id,classification_type,transformation_type,block_group_id,processing_date,country_code,stop_date,stop_hour,max_distance_from_home_meters,local_date,total_home_dwell_time_minutes,home_known_flag,work_known_flag,home_admin1_id,home_admin2_id,home_block_group_id,work_admin1_id,work_admin2_id
0,5709141442,GAID,41.608207,-87.792550,dp3sdnx3,2,0.000000,18.4590,0.000000,21.366667,2022-08-01T05:47:35-05:00,-18000,US.IL,US.IL.031,WHITELISTED,KEEP,US.IL.031.170318245063,20220803,US,2022-08-01,5,4.796742e+03,20220801,703.166667,True,False,US.IL,US.IL.031,170318241051,None,None
1,5604556850,GAID,41.887719,-87.627466,dp3wmcrt,5,31.849941,97.7940,15.785645,17.583333,2022-08-01T22:54:54-05:00,-18000,US.IL,US.IL.031,WHITELISTED,KEEP,US.IL.031.170313201002,20220803,US,2022-08-01,22,1.479916e+06,20220801,0.000000,True,False,US.CO,US.CO.031,080310020001,None,None
2,5604556850,GAID,41.888169,-87.626921,dp3wq180,2,0.000000,98.4000,0.000000,35.400000,2022-08-01T23:48:27-05:00,-18000,US.IL,US.IL.031,WHITELISTED,KEEP,US.IL.031.170310815003,20220803,US,2022-08-01,23,1.479916e+06,20220801,0.000000,True,False,US.CO,US.CO.031,080310020001,None,None
3,5604556850,GAID,41.974610,-87.653814,dp3wv3hr,4,18.900131,14.7425,3.757973,136.450000,2022-08-01T18:15:49-05:00,-18000,US.IL,US.IL.031,WHITELISTED,KEEP,US.IL.031.170310313005,20220803,US,2022-08-01,18,1.479916e+06,20220801,0.000000,True,False,US.CO,US.CO.031,080310020001,None,None
4,5477292498,GAID,41.905017,-87.746970,dp3w7k8d,2,0.000000,18.9100,0.000000,40.416667,2022-08-01T12:59:53-05:00,-18000,US.IL,US.IL.031,WHITELISTED,KEEP,US.IL.031.170318313001,20220803,US,2022-08-01,12,1.131473e+04,20220801,470.450000,True,False,US.IL,US.IL.031,170310503003,None,None


<font size=5>Combine "filtered_stop_data" and "Demographic data" into together</font>  
<font color='orange'>For successfully running the example coding as below(based on the final function code order), we have to combine "filtered_stop_data" and "Demographic data" into together now.

In [31]:
# create two new columns in "filtered_stop_data" dataset: "home_tract_id" and "tract_id"
filtered_stop_data_test['home_tract_id'] = filtered_stop_data_test['home_block_group_id'].apply(lambda x: x[:-1])
filtered_stop_data_test['tract_id'] = filtered_stop_data_test['block_group_id'].apply(lambda x: x[:-1])

# create the "stop_and_census_data" dataframe by merge "filtered_stop_data" and "cw_2010_2020" dataframe
stop_and_census_data_test = pd.merge(filtered_stop_data_test, cw_2010_2020, how="inner", left_on="home_tract_id", right_on="home_tract_id")

stop_and_census_data_test.info()
stop_and_census_data_test.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497020 entries, 0 to 497019
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   cuebiq_id                      497020 non-null  int64  
 1   device_type_code               497020 non-null  object 
 2   lat                            497020 non-null  float64
 3   lng                            497020 non-null  float64
 4   geohash_id                     497020 non-null  object 
 5   cluster_size                   497020 non-null  int64  
 6   avg_distance_meters            497020 non-null  float64
 7   avg_accuracy_meters            497016 non-null  float64
 8   std_accuracy_meters            497016 non-null  float64
 9   dwell_time_minutes             497020 non-null  float64
 10  stop_zoned_datetime            497020 non-null  object 
 11  timezone_offset_seconds        497020 non-null  int64  
 12  admin1_id                     

,cuebiq_id,device_type_code,lat,lng,geohash_id,cluster_size,avg_distance_meters,avg_accuracy_meters,std_accuracy_meters,dwell_time_minutes,stop_zoned_datetime,timezone_offset_seconds,admin1_id,admin2_id,classification_type,transformation_type,block_group_id,processing_date,country_code,stop_date,stop_hour,max_distance_from_home_meters,local_date,total_home_dwell_time_minutes,home_known_flag,work_known_flag,home_admin1_id,home_admin2_id,home_block_group_id,work_admin1_id,work_admin2_id,home_tract_id,tract_id,gisjoin,hispanic,total_pop,white,black,indigenous,asian,pac_isl,other,two_more,ba,ma,prof,phd,median_income,total_school_pop,state,county,tract,weights
0,5709141442,GAID,41.608207,-87.792550,dp3sdnx3,2,0.000000,18.459000,0.000000,21.366667,2022-08-01T05:47:35-05:00,-18000,US.IL,US.IL.031,WHITELISTED,KEEP,US.IL.031.170318245063,20220803,US,2022-08-01,5,4796.742323,20220801,703.166667,True,False,US.IL,US.IL.031,170318241051,None,None,17031824105,US.IL.031.17031824506,G1700310824105,567.0,7269.0,5888.0,99.0,0.0,593.0,0.0,122.00001,122.00001,1377.0,813.0,114.0,59.0,102859.01,789.99994,17,031,824105,8.297945
1,5450574473,IDFA,41.639980,-87.904651,dp3kzsmf,5,2.840489,5.120000,0.672309,3.033333,2022-08-01T21:10:22-05:00,-18000,US.IL,US.IL.031,OTHER,KEEP,US.IL.031.170318241133,20220802,US,2022-08-01,21,6482.823343,20220801,1093.950000,True,False,US.IL,US.IL.031,170318241051,None,None,17031824105,US.IL.031.17031824113,G1700310824105,567.0,7269.0,5888.0,99.0,0.0,593.0,0.0,122.00001,122.00001,1377.0,813.0,114.0,59.0,102859.01,789.99994,17,031,824105,8.297945
2,5450574473,IDFA,41.652624,-87.856962,dp3sby8s,7,15.007033,7.371429,5.163886,3.666667,2022-08-01T21:22:39-05:00,-18000,US.IL,US.IL.031,WHITELISTED,KEEP,US.IL.031.170318238012,20220802,US,2022-08-01,21,6482.823343,20220801,1093.950000,True,False,US.IL,US.IL.031,170318241051,None,None,17031824105,US.IL.031.17031823801,G1700310824105,567.0,7269.0,5888.0,99.0,0.0,593.0,0.0,122.00001,122.00001,1377.0,813.0,114.0,59.0,102859.01,789.99994,17,031,824105,8.297945
3,5351008871,IDFA,41.609280,-87.847645,dp3s8yxr,4,7.370786,16.100000,14.518264,31.850000,2022-08-01T09:52:35-05:00,-18000,US.IL,US.IL.031,WHITELISTED,KEEP,US.IL.031.170318241063,20220801,US,2022-08-01,9,5675.115823,20220801,1097.033333,True,False,US.IL,US.IL.031,170318241051,None,None,17031824105,US.IL.031.17031824106,G1700310824105,567.0,7269.0,5888.0,99.0,0.0,593.0,0.0,122.00001,122.00001,1377.0,813.0,114.0,59.0,102859.01,789.99994,17,031,824105,8.297945
4,5351008871,IDFA,41.631941,-87.853451,dp3sbfg5,16,10.522570,4.668750,0.060208,10.583333,2022-08-01T12:37:27-05:00,-18000,US.IL,US.IL.031,WHITELISTED,KEEP,US.IL.031.170318241051,20220801,US,2022-08-01,12,5675.115823,20220801,1097.033333,True,False,US.IL,US.IL.031,170318241051,None,None,17031824105,US.IL.031.17031824105,G1700310824105,567.0,7269.0,5888.0,99.0,0.0,593.0,0.0,122.00001,122.00001,1377.0,813.0,114.0,59.0,102859.01,789.99994,17,031,824105,8.297945


## 3.The function of `'get_tract_sizes'`   

**Explanation:**  
This function computed tract sizes for different regions based on the count of the same "home_tract_id" from "filtered_stop_data" and specific "county names"  
- `get_tract_sizes`: input one "filtered_stop_data" dataframe at one county  
- `get_tract_sizes_many`: input one "filtered_stop_data" dataframe at all counties

In [32]:
def get_tract_sizes(filtered_stop_data, county):
    filtered = filtered_stop_data.groupby('home_tract_id').count()[['cuebiq_id']]
    filtered.reset_index(inplace=True)
    filtered['home_tract_id'] = filtered['home_tract_id'].apply(lambda x: county + '.' + x)
    return filtered
def get_tract_sizes_many(filtered_stop_data):
    filtered = filtered_stop_data.groupby('home_tract_id').agg({'cuebiq_id':'count', 'admin2_id':'first'})[['cuebiq_id', 'admin2_id']]
    filtered.reset_index(inplace=True)
    filtered['home_tract_id'] = filtered.apply(lambda x: x['admin2_id'] + '.' + x['home_tract_id'], axis=1)
    return filtered[['home_tract_id','cuebiq_id']]

<font color='blue'>**For Example:   
function <u>"3:get_tract_sizes"</u>**</font>

In [33]:
# create the "tract_sizes"(population) dataframe from "filtered_stop_data" and "given county" by "3.1.15 get_tract_sizes" function
tract_sizes_test = get_tract_sizes(filtered_stop_data_test, county)
tract_sizes_test.info()
tract_sizes_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12168 entries, 0 to 12167
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   home_tract_id  12168 non-null  object
 1   cuebiq_id      12168 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 190.2+ KB


,home_tract_id,cuebiq_id
0,US.IL.031.01003010703,2
1,US.IL.031.01003010800,8
2,US.IL.031.01003011300,1
3,US.IL.031.01003011401,1
4,US.IL.031.01003011405,4


<font color='Orange'>The function of **<u>4,5,6,7,8</u>** are combined into one example.</font>  
<font color='Orange' size=5><u>and finally, the function of 10 is combine **<u>4,5,6,7,8,9</u>** into together.</u></font>

## 4.The function of `'add_race_freqs'`  
**Explanation:**  
This function add given "race_freq" list and "cell" together.  
- If "cell" value is Null, it will directly return to input "race_freq"
- If "cell" value is not Null, it will add the "race_freq" list and  the "cell" list together

In [34]:
def add_race_freqs(cell, race_freq):
    """
    update the cell.
    if it is None return the race_freq.
    if it already has a list of race frequencies:
    [WHITE_0, BLACK_0, ASIAN_0, AMERICAN_INDIAN_0, NATIVE_HAWAIIAN_0, HISPANIC_0]
    and we would like to add the race frequencies for another device:
    [WHITE, BLACK, ASIAN, AMERICAN_INDIAN, NATIVE_HAWAIIAN, HISPANIC]
    
    return this [WHITE_0 + WHITE, BLACK_0 + BLACK, ASIAN_0 + ASIAN, ...]
    """
    if cell is None:
        return race_freq
    else:
        return np.add(cell, race_freq)

## 5.The function of "update_time_overlap_df"  
**Explanation:**  
This function add the race frequencies in the stop data rows to each time bucket within a time period and update the values of these time buckets accordingly in the "time_overlap_df" dataframe.  
- `add = lambda x : add_race_freqs(x, race_freq)`: Update race frequencies from "race_freq" dataframe by **4. "add_race_freqs" function**

In [35]:
TOTAL_POP = 'total_pop'
OTHER = "other"
WHITE = "white"
BLACK = "black"
ASIAN = "asian"
HISPANIC = "hispanic"

## TO DO: INCLUDE TOTAL POPULATION BECAUSE ULTIMATELY DEMONIMATOR HAS TO BE THAT...I THINK 
## N = total population
def update_time_overlap_df(stop_data_row, time_overlap_df, time_buckets):
    """
    this function is applied to each row in stop data. 
    
    it adds the race frequencies of the stop_data_row to each time bucket between the start index and end index.
    
    updates time_overlap_df
    """
    
    race_list = [WHITE, BLACK, ASIAN, HISPANIC, OTHER,TOTAL_POP]
    
    index_start = stop_data_row["start_bucket_index"]
    index_end = stop_data_row["end_bucket_index"]
    cuebiq_id = stop_data_row["cuebiq_id"]
    geohash_id = stop_data_row["geohash_id"]
    num_buckets = len(time_buckets)
    
    # because the [index_start:index_end] is front closed and black open(for example,[3:5], only can choose 3,4),to ensure all value can be choose,we have to update index_end into index_end + 1
    index_end = min(index_end + 1, num_buckets) 
    
    race_freq = stop_data_row[race_list].values
#     race_freq = np.insert(race_freq, 4, race_freq[4]-(np.sum(race_freq[0:4])))
    race_freq = np.append(race_freq, 1)
    
    # multiply race_freq by weight HERE
    add = lambda x : add_race_freqs(x, race_freq)
    time_overlap_df.loc[geohash_id].iloc[index_start: index_end] = time_overlap_df.loc[geohash_id].iloc[index_start: index_end].apply(add)   
#     print(time_overlap_df.loc[geohash_id].iloc[index_start: index_end])

## 6.The function of `'get_geohash_ids' ` 
**Explanation:**  
This function create the new dataframe("geohash_ids") for storing the "geohash_id" column from <font color='blue'><u>**stop_data**</u></font> DB.

In [36]:
def get_geohash_ids(stop_data):
    """
    stop_data: pandas dataframe 
    
    returns a set of the geohash_ids in the stop_data dataframe (no duplicates)
    """
    
    geohash_ids = set()
    for geohash in stop_data["geohash_id"]:
        geohash_ids.add(geohash)
        
    return geohash_ids

<font color='blue'>**For Example:   
function "6.The function of 'get_geohash_ids'"**</font>

In [37]:
geohash_list_test = list(get_geohash_ids(stop_data_test))
print(geohash_list_test[:5])
print("Number of elements in geohash_list_test:", len(geohash_list_test))

['dp3t5f5m', 'dp3qfwsc', 'dp3r3ydq', 'dp3tvjkq', 'dp3sxhrr']
Number of elements in geohash_list_test: 242300


## 7.The function of `'create_time_buckets'` 
**Explanation:**  
This function create the time_buckets based on specific <font color='blue'> <u>**"dataframe"**</u>, <u>**"date"**</u> and **<u>"bucket_size"(15 mins)**</u></font>

In [38]:
def create_time_buckets(stop_data, bucket_size, date):
    """
    stop_data: pandas dataframe
    bucket_size: int
    
    returns a dataframe of the start times of each bucket.  
    the size of each time bucket will be determine by the bucket_size argument 
    """
    
    # get the min and max stop times 
    # stop_data["stop_zoned_datetime"] = pd.to_datetime(stop_data["stop_zoned_datetime"])
    date_str = date.strftime('%Y-%m-%d')
    min_time = pd.to_datetime(date_str + " 12:00:00AM")
    max_time = pd.to_datetime(date_str + " 11:59:59PM")
    
    # find the number of minutes between the min and max stop times 
    difference = max_time - min_time
    difference_in_mins = difference.total_seconds() / 60
    print("Total difference between max and min time in minutes: ", difference_in_mins)
    
    # calculate the number of buckets by dividing the total minutes within this interval: 
    # [min_stop_time, max_stop_time] by the bucket_size
    num_buckets = int(difference_in_mins / bucket_size) + 1
    print("Total number of buckets: ", num_buckets)
    
    # create a dataframe of all the time periods 
    buckets = pd.date_range(start=min_time, end=max_time, periods=num_buckets)
    buckets_df = buckets.to_frame(index=False, name="time")
    
    return buckets_df["time"].tolist()

<font color='blue'>**For Example:   
function "7.The function of 'create_time_buckets'"**</font>

In [39]:
time_buckets_test = create_time_buckets(stop_data_test, 15, date)
print(time_buckets_test[:5])
print("Number of elements in time_buckets_test:", len(time_buckets_test))

Total difference between max and min time in minutes:  1439.9833333333333
Total number of buckets:  96
[Timestamp('2022-08-01 00:00:00'), Timestamp('2022-08-01 00:15:09.463157894'), Timestamp('2022-08-01 00:30:18.926315789'), Timestamp('2022-08-01 00:45:28.389473684'), Timestamp('2022-08-01 01:00:37.852631578')]
Number of elements in time_buckets_test: 96


## 8.The Function of `"find_time_overlap"`  

**Explanation:**  
This function create the <font color='blue'> <u>**start_bucket_index**</u>, **<u>end_bucket_index**</u></font> columns and  <font color='blue'> <u>**time_overlap_df**</u></font> dateframe for **5. "update_time_overlap_df" function**, and finally create the new <font color='blue'><u>**"time_overlap_df"**</u></font> dataframe ro storage the race frequency in each unit.

In [40]:
def find_time_overlap(stop_data, bucket_size, date):
    """
    stop_data: pandas dataframe containing stop data 
    bucket_size: int that corresponds to the time interval between each bucket (in minutes)
    
    returns a pandas dataframe with cols that correspond to time_buckets, and rows that correspond to geohash_ids.
    each cell in the dataframe is either None if there were no stops in that particular block group in that time bucket, 
    or it is a list of the following format:
    [WHITE, BLACK, ASIAN, HISPANIC, OTHER, NUM_PPL] 
    with each element being the frequency of that specific race in each device's block_group / tract
    """

    geohash_list = list(get_geohash_ids(stop_data))
    time_overlap_df = pd.DataFrame(index=geohash_list)
    
    time_buckets = create_time_buckets(stop_data, bucket_size, date)  
    for bucket in time_buckets:
        time_overlap_df[bucket] = None
    
    ## New Stop Time
    stop_time = stop_data["stop_zoned_datetime"].apply(lambda x: pd.to_datetime(x,utc=True) - pd.DateOffset(hours=int(str(x)[-4:-3])))
    min_stop_time = stop_time.dt.floor("D")


    # min_stop_time = stop_and_census_data.apply(lambda x: x['min_stop_time'] -pd.DateOffset(hours=16)

    difference_start = (stop_time - min_stop_time).dt.total_seconds()
    difference_start_in_mins = difference_start/ 60

    stop_data["start_bucket_index"] = (difference_start_in_mins / bucket_size).astype(int)  
    # calculate the index of the time bucket where the stop ended 
    dwell_time = stop_data["dwell_time_minutes"]
    difference_end_in_mins = difference_start_in_mins + dwell_time
    stop_data["end_bucket_index"] =np.minimum(96,(difference_end_in_mins / bucket_size).astype(int))

    stop_data.apply(update_time_overlap_df, args=(time_overlap_df, time_buckets), axis=1)
    return time_overlap_df

In [41]:
# def calculate_exposure_element(race_freq):
#     """
#     for a list of race frequecies which looks like this:
#     [WHITE, BLACK, ASIAN, HISPANIC, OTHER] 
#     calculate the simpson's index using this formula:
    
#     numerator = sum[n(n-1)] where n is each element of the list 
#     denominator = N(N-1) where N is the sum of all elements in the list 
#     """
#     y = race_freq
#     if race_freq is None:
#         return 0
    
#     num_ppl = race_freq[-1]
#     N = race_freq[-2]
#     race_freq = race_freq[:-2]
    
#     numerator = (race_freq * (race_freq - 1)).sum()
#     denominator = N * (N - 1)
    
#     if numerator == 0 or denominator == 0:
#         return 0
    
#     s_index = 1 - (numerator/denominator)
#     if s_index < 0:
#         print(y)
#     return (s_index, num_ppl)

<font color='blue'>**For Example:   
function "8.find_time_overlap"**</font>

In [42]:
# to record the time when the program starts executing
exposure_start_time = time.time()

overlap_df_test = find_time_overlap(stop_and_census_data_test, 15, date)

# to record the time when the program finish executing
exposure_end_time = time.time()
# calculate the total executing time
total_time = exposure_end_time - exposure_start_time
# print the total executing time
print(f"total executing time: {total_time} seconds")

overlap_df_test.info()
overlap_df_test.head()

Total difference between max and min time in minutes:  1439.9833333333333
Total number of buckets:  96
total executing time: 476.6359062194824 seconds
<class 'pandas.core.frame.DataFrame'>
Index: 239731 entries, dp3qfwsc to dp3wtpf9
Data columns (total 96 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   2022-08-01 00:00:00            1365 non-null   object
 1   2022-08-01 00:15:09.463157894  2121 non-null   object
 2   2022-08-01 00:30:18.926315789  2471 non-null   object
 3   2022-08-01 00:45:28.389473684  2786 non-null   object
 4   2022-08-01 01:00:37.852631578  2938 non-null   object
 5   2022-08-01 01:15:47.315789473  3021 non-null   object
 6   2022-08-01 01:30:56.778947368  3132 non-null   object
 7   2022-08-01 01:46:06.242105263  3206 non-null   object
 8   2022-08-01 02:01:15.705263157  3312 non-null   object
 9   2022-08-01 02:16:25.168421052  3359 non-null   object
 10  2022-08-01 02:31:34.6

,2022-08-01 00:00:00.000000000,2022-08-01 00:15:09.463157894,2022-08-01 00:30:18.926315789,2022-08-01 00:45:28.389473684,2022-08-01 01:00:37.852631578,2022-08-01 01:15:47.315789473,2022-08-01 01:30:56.778947368,2022-08-01 01:46:06.242105263,2022-08-01 02:01:15.705263157,2022-08-01 02:16:25.168421052,2022-08-01 02:31:34.631578947,2022-08-01 02:46:44.094736842,2022-08-01 03:01:53.557894736,2022-08-01 03:17:03.021052631,2022-08-01 03:32:12.484210526,2022-08-01 03:47:21.947368421,2022-08-01 04:02:31.410526315,2022-08-01 04:17:40.873684210,2022-08-01 04:32:50.336842105,2022-08-01 04:47:59.800000000,2022-08-01 05:03:09.263157894,2022-08-01 05:18:18.726315789,2022-08-01 05:33:28.189473684,2022-08-01 05:48:37.652631578,2022-08-01 06:03:47.115789473,2022-08-01 06:18:56.578947368,2022-08-01 06:34:06.042105263,2022-08-01 06:49:15.505263157,2022-08-01 07:04:24.968421052,2022-08-01 07:19:34.431578947,2022-08-01 07:34:43.894736842,2022-08-01 07:49:53.357894736,2022-08-01 08:05:02.821052631,2022-08-01 08:20:12.284210526,2022-08-01 08:35:21.747368421,2022-08-01 08:50:31.210526315,2022-08-01 09:05:40.673684210,2022-08-01 09:20:50.136842105,2022-08-01 09:35:59.600000000,2022-08-01 09:51:09.063157894,2022-08-01 10:06:18.526315789,2022-08-01 10:21:27.989473684,2022-08-01 10:36:37.452631578,2022-08-01 10:51:46.915789473,2022-08-01 11:06:56.378947368,2022-08-01 11:22:05.842105263,2022-08-01 11:37:15.305263157,2022-08-01 11:52:24.768421052,2022-08-01 12:07:34.231578947,2022-08-01 12:22:43.694736842,2022-08-01 12:37:53.157894736,2022-08-01 12:53:02.621052631,2022-08-01 13:08:12.084210526,2022-08-01 13:23:21.547368421,2022-08-01 13:38:31.010526315,2022-08-01 13:53:40.473684210,2022-08-01 14:08:49.936842105,2022-08-01 14:23:59.400000000,2022-08-01 14:39:08.863157894,2022-08-01 14:54:18.326315789,2022-08-01 15:09:27.789473684,2022-08-01 15:24:37.252631578,2022-08-01 15:39:46.715789473,2022-08-01 15:54:56.178947368,2022-08-01 16:10:05.642105263,2022-08-01 16:25:15.105263157,2022-08-01 16:40:24.568421052,2022-08-01 16:55:34.031578947,2022-08-01 17:10:43.494736842,2022-08-01 17:25:52.957894736,2022-08-01 17:41:02.421052631,2022-08-01 17:56:11.884210526,2022-08-01 18:11:21.347368421,2022-08-01 18:26:30.810526315,2022-08-01 18:41:40.273684210,2022-08-01 18:56:49.736842105,2022-08-01 19:11:59.200000000,2022-08-01 19:27:08.663157894,2022-08-01 19:42:18.126315789,2022-08-01 19:57:27.589473684,2022-08-01 20:12:37.052631578,2022-08-01 20:27:46.515789473,2022-08-01 20:42:55.978947368,2022-08-01 20:58:05.442105263,2022-08-01 21:13:14.905263157,2022-08-01 21:28:24.368421052,2022-08-01 21:43:33.831578947,2022-08-01 21:58:43.294736842,2022-08-01 22:13:52.757894736,2022-08-01 22:29:02.221052631,2022-08-01 22:44:11.684210526,2022-08-01 22:59:21.147368421,2022-08-01 23:14:30.610526315,2022-08-01 23:29:40.073684210,2022-08-01 23:44:49.536842105,2022-08-01 23:59:59.000000000
dp3qfwsc,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"[1836.0, 65.0, 1135.0, 148.0, 352.0, 3535.9998...","[1836.0, 65.0, 1135.0, 148.0, 352.0, 3535.9998...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
dp3tvjkq,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"[2926.0, 1.0, 64.0, 2327.0, 76.00000399999999,...","[2926.0, 1.0, 64.0, 2327.0, 76.00000399999999,...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Non

In [43]:
print(overlap_df_test.loc['dp3tvt93','2022-08-01 07:49:53.357894736'])
print(overlap_df_test.loc['dp3t7kht','2022-08-01 00:00:00.000000000'])

[1544.0 36.0 425.0 236.0 42.000004 2283.0 1]
[6139.9998 1100.0 456.0 5042.0 673.0 13410.9998 3]


## 9.The Function of `"calculate_exposure_per_group_element"`  

**Explanation:**  
Calculate Simpson's index from the input list of racial frequencies of the "overlap_df" dataframe.

In [44]:
def calculate_exposure_per_group_element(race_freq):
    """
    for a list of race frequencies which looks like this:
    [WHITE, BLACK, ASIAN,HISPANIC,OTHER] 
    calculate the simpson's index using this formula:
    
    numerator = sum[n(n-1)] where n is each element of the list 
    denominator = N(N-1) where N is the sum of all elements in the list 
    
    exposure conditional on there being more than one person
    """
    y = race_freq
    if race_freq is None:
        return 0
    else: 
        if race_freq[-1]<2:
            return 0
        else: 
            num_ppl = race_freq[-1]
            N = race_freq[-2]
            race_freq = race_freq[:-2]
            numerator_w = race_freq[0]*(N-race_freq[0])
#             print(race_freq)
            numerator_b = race_freq[1]*(N-race_freq[1])
            numerator_a = race_freq[2]*(N-race_freq[2])
            numerator_h = race_freq[3]*(N-race_freq[3])
            numerator_o = race_freq[4]*(N-race_freq[4])
            
            numerator = np.array([numerator_w,numerator_b,numerator_a,numerator_h,numerator_o])
#             numerator = race_freq * (N - race_freq)
            denominator = N * (N - 1)

            group_diversity = numerator/denominator

        return (group_diversity, num_ppl)

In [45]:
def calculate_exposure_df(overlap_df):
    """
    calculate the simpson's index for each cell of the overlap dataframe
    
    returns a new dataframe where each cell contains a simpson's index
    """
    
    exposure = overlap_df.applymap(calculate_exposure_per_group_element)
    
    return exposure

<font color='blue'>**For Example:   
function "9.calculate_exposure_per_group_element"**</font>

In [46]:
exposure_test = overlap_df_test.applymap(calculate_exposure_per_group_element)
exposure_test.info()
exposure_test.head()

<class 'pandas.core.frame.DataFrame'>
Index: 239731 entries, dp3qfwsc to dp3wtpf9
Data columns (total 96 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   2022-08-01 00:00:00            239731 non-null  object
 1   2022-08-01 00:15:09.463157894  239731 non-null  object
 2   2022-08-01 00:30:18.926315789  239731 non-null  object
 3   2022-08-01 00:45:28.389473684  239731 non-null  object
 4   2022-08-01 01:00:37.852631578  239731 non-null  object
 5   2022-08-01 01:15:47.315789473  239731 non-null  object
 6   2022-08-01 01:30:56.778947368  239731 non-null  object
 7   2022-08-01 01:46:06.242105263  239731 non-null  object
 8   2022-08-01 02:01:15.705263157  239731 non-null  object
 9   2022-08-01 02:16:25.168421052  239731 non-null  object
 10  2022-08-01 02:31:34.631578947  239731 non-null  object
 11  2022-08-01 02:46:44.094736842  239731 non-null  object
 12  2022-08-01 03:01:53.557894736  239731 no

,2022-08-01 00:00:00.000000000,2022-08-01 00:15:09.463157894,2022-08-01 00:30:18.926315789,2022-08-01 00:45:28.389473684,2022-08-01 01:00:37.852631578,2022-08-01 01:15:47.315789473,2022-08-01 01:30:56.778947368,2022-08-01 01:46:06.242105263,2022-08-01 02:01:15.705263157,2022-08-01 02:16:25.168421052,2022-08-01 02:31:34.631578947,2022-08-01 02:46:44.094736842,2022-08-01 03:01:53.557894736,2022-08-01 03:17:03.021052631,2022-08-01 03:32:12.484210526,2022-08-01 03:47:21.947368421,2022-08-01 04:02:31.410526315,2022-08-01 04:17:40.873684210,2022-08-01 04:32:50.336842105,2022-08-01 04:47:59.800000000,2022-08-01 05:03:09.263157894,2022-08-01 05:18:18.726315789,2022-08-01 05:33:28.189473684,2022-08-01 05:48:37.652631578,2022-08-01 06:03:47.115789473,2022-08-01 06:18:56.578947368,2022-08-01 06:34:06.042105263,2022-08-01 06:49:15.505263157,2022-08-01 07:04:24.968421052,2022-08-01 07:19:34.431578947,2022-08-01 07:34:43.894736842,2022-08-01 07:49:53.357894736,2022-08-01 08:05:02.821052631,2022-08-01 08:20:12.284210526,2022-08-01 08:35:21.747368421,2022-08-01 08:50:31.210526315,2022-08-01 09:05:40.673684210,2022-08-01 09:20:50.136842105,2022-08-01 09:35:59.600000000,2022-08-01 09:51:09.063157894,2022-08-01 10:06:18.526315789,2022-08-01 10:21:27.989473684,2022-08-01 10:36:37.452631578,2022-08-01 10:51:46.915789473,2022-08-01 11:06:56.378947368,2022-08-01 11:22:05.842105263,2022-08-01 11:37:15.305263157,2022-08-01 11:52:24.768421052,2022-08-01 12:07:34.231578947,2022-08-01 12:22:43.694736842,2022-08-01 12:37:53.157894736,2022-08-01 12:53:02.621052631,2022-08-01 13:08:12.084210526,2022-08-01 13:23:21.547368421,2022-08-01 13:38:31.010526315,2022-08-01 13:53:40.473684210,2022-08-01 14:08:49.936842105,2022-08-01 14:23:59.400000000,2022-08-01 14:39:08.863157894,2022-08-01 14:54:18.326315789,2022-08-01 15:09:27.789473684,2022-08-01 15:24:37.252631578,2022-08-01 15:39:46.715789473,2022-08-01 15:54:56.178947368,2022-08-01 16:10:05.642105263,2022-08-01 16:25:15.105263157,2022-08-01 16:40:24.568421052,2022-08-01 16:55:34.031578947,2022-08-01 17:10:43.494736842,2022-08-01 17:25:52.957894736,2022-08-01 17:41:02.421052631,2022-08-01 17:56:11.884210526,2022-08-01 18:11:21.347368421,2022-08-01 18:26:30.810526315,2022-08-01 18:41:40.273684210,2022-08-01 18:56:49.736842105,2022-08-01 19:11:59.200000000,2022-08-01 19:27:08.663157894,2022-08-01 19:42:18.126315789,2022-08-01 19:57:27.589473684,2022-08-01 20:12:37.052631578,2022-08-01 20:27:46.515789473,2022-08-01 20:42:55.978947368,2022-08-01 20:58:05.442105263,2022-08-01 21:13:14.905263157,2022-08-01 21:28:24.368421052,2022-08-01 21:43:33.831578947,2022-08-01 21:58:43.294736842,2022-08-01 22:13:52.757894736,2022-08-01 22:29:02.221052631,2022-08-01 22:44:11.684210526,2022-08-01 22:59:21.147368421,2022-08-01 23:14:30.610526315,2022-08-01 23:29:40.073684210,2022-08-01 23:44:49.536842105,2022-08-01 23:59:59.000000000
dp3qfwsc,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
dp3tvjkq,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
dp3t5f5m,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
dp3r3ydq,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
dp3wukm6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [47]:
# statistic the number of rows if there is any cell is not equal to 0 in current row.
non_zero_rows_count = exposure_test.apply(lambda row: any(cell != 0 for cell in row), axis=1).sum()
non_zero_rows_count

39780

In [48]:
print(exposure_test.loc['dp3qzcrv','2022-08-01 07:19:34.431578947'])

(array([4.70057212e-02, 1.24877867e-02, 1.03694281e-05, 2.56530932e-02,
       1.03363269e-02]), 2)


## 10.The Function of `"get_exposure"`  

**Explanation:**  
This function combined the <font color='blue'><u>**function 8**</u> and <u>**9**</u></font> together, and the result should be the same with above. **<u>Basically, it calculate the Simpson's index for each geohash id and time brackets.</u>** 

In [49]:
def get_exposure(data, date):
    overlap_df = find_time_overlap(data, 15, date)
    exposure = calculate_exposure_df(overlap_df)
    
    return exposure

<font color='Orange' size=5>**Updated coding**:   
the function of <u>11,12</u> are combine into **separately** calculating the **exposures** in 5 time periods.</font>

## 11.The Function of `"average_exposures"`</font>  

**Explanation:**  
This function calculate the "average_exposures" based on "weighted_exposure" and "total_weight" in each geohash_id.

In [50]:
def average_exposures(row):
    """
    For each geohash in the exposures table,
    calculate the weighted exposure based on how many people are in each geohash
    """
    
    row = row.values
    row = row[row != 0]
    
    if len(row) == 0:
#         return 0
        return [0,0,0,0,0]
    else: 
        weighted_exposure = sum(i * j for i, j in row)
        total_weight = sum(j for i, j in row)

        avg = weighted_exposure/total_weight
        return avg

## <font color='red'>12.The Function of `"aggregate_time_intervals"`</font>  

**Explanation:**  
This function <font color='blue'>**separately**</font> calculate the <font color='blue'><u>**"aggregate_exposures"</u>**</font> in 5 group time  
- late night：[0,6)  
- morning：[6,12)  
- afternoon：[12,17)  
- evening：[17,22)  
- late evening：[22,24)  

In [51]:
time_intervals = [('late night', 0, 6),
                  ('morning', 6, 12),
                  ('afternoon', 12, 17),
                  ('evening', 17, 22),
                  ('late evening', 22, 24)
                 ]

In [52]:
def aggregate_time(exposure_table):
    aggregated_exposures = exposure_table.apply(lambda x: average_exposures(x), axis=1)
    white =  np.array(aggregated_exposures.tolist())[:,0]
    black =  np.array(aggregated_exposures.tolist())[:,1]
    asian =  np.array(aggregated_exposures.tolist())[:,2]
    hispanic =  np.array(aggregated_exposures.tolist())[:,3]
    other = np.array(aggregated_exposures.tolist())[:,4]
    
    aggregated_exposures_df = pd.DataFrame({
        'geohash_id':aggregated_exposures.index, 
        'white_exposure': white,
        'black_exposure': black,
        'asian_exposure': asian,
        'hispanic_exposure': hispanic,
        'other_exposure': other
    }, dtype=float)
    
    return aggregated_exposures_df

def aggregate_time_intervals(exposure_table, time_intervals, date):
    timestamp_intervals = []

    for label, start_hour, end_hour in time_intervals:
        start_timestamp = pd.Timestamp(f"{date} {start_hour}:00:00")
        end_timestamp = pd.Timestamp(f"{date} {end_hour-1}:59:59")
        timestamp_intervals.append((label, start_timestamp, end_timestamp))

    new_dfs = {}
    for label, start, end in timestamp_intervals:
        # Extract columns within the time range
        columns_in_range = exposure_table.columns[(exposure_table.columns >= start) & (exposure_table.columns < end)]

        # Create a new DataFrame for the time range
        new_dfs[label] = exposure_table[columns_in_range]
    
    interval_table_arr = {}
    for label, table in new_dfs.items():
        aggregated_exposures = table.apply(lambda x: average_exposures(x), axis=1)
        white =  np.array(aggregated_exposures.tolist())[:,0]
        black =  np.array(aggregated_exposures.tolist())[:,1]
        asian =  np.array(aggregated_exposures.tolist())[:,2]
        hispanic =  np.array(aggregated_exposures.tolist())[:,3]
        other = np.array(aggregated_exposures.tolist())[:,4]
        interval_table_arr[label] = pd.DataFrame({
            'geohash_id':aggregated_exposures.index, 
            'white_exposure': white,
            'black_exposure': black,
            'asian_exposure': asian,
            'hispanic_exposure': hispanic,
            'other_exposure': other
        })
    
    return interval_table_arr

<font color='blue'>**For Example:   
function "12.aggregate_time_intervals"**</font>

In [53]:
# to record the time when the program starts executing
exposure_start_time = time.time()

time_aggregate_test = aggregate_time_intervals(exposure_test, time_intervals, date)

# to record the time when the program finish executing
print("total executing time", (time.time() - exposure_start_time)/60, "minutes to run")

total executing time 0.28209488789240517 minutes to run


In [54]:
test = pd.DataFrame()

for time_period, df in time_aggregate_test.items():
    # create a new column represent the time period
    df['time_period'] = time_period
    # contact the 5 timeinterval periods into together
    test = pd.concat([test, df], ignore_index=True)

test.info()
test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198655 entries, 0 to 1198654
Data columns (total 7 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   geohash_id         1198655 non-null  object 
 1   white_exposure     1198655 non-null  float64
 2   black_exposure     1198655 non-null  float64
 3   asian_exposure     1198655 non-null  float64
 4   hispanic_exposure  1198655 non-null  float64
 5   other_exposure     1198655 non-null  float64
 6   time_period        1198655 non-null  object 
dtypes: float64(5), object(2)
memory usage: 64.0+ MB


,geohash_id,white_exposure,black_exposure,asian_exposure,hispanic_exposure,other_exposure,time_period
0,dp3qfwsc,0.0,0.0,0.0,0.0,0.0,late night
1,dp3tvjkq,0.0,0.0,0.0,0.0,0.0,late night
2,dp3t5f5m,0.0,0.0,0.0,0.0,0.0,late night
3,dp3r3ydq,0.0,0.0,0.0,0.0,0.0,late night
4,dp3wukm6,0.0,0.0,0.0,0.0,0.0,late night


## 13.The Function of `"crosswalk_tract_geohash"`  

**Explanation:**    
This function create the weight in different tract, which represents the "stay" degree.  
- Step 1: find the total number of stays per `"geohash id"` 
- Step 2: find the total number of stays per `"geohash id and tract id"`
- Step 3: weight= (2) divided by (1)

In [55]:
def crosswalk_blkgrp_geohash(data):
    """
    output a mapping between geohash ids and block group ids. 
    assign a weight to those mappings using the following method:
    (1) find the total number of stays per geohash id 
    (2) find the total number of stays per geohash id and block group 
    the weight is equal to (2) divided by (1)
    """
    
    data = data[['geohash_id', 'block_group_id']]
    total_stays_in_geohash = data.groupby('geohash_id')['block_group_id'].count().reset_index(name="total_stays_in_geohash")
    total_stays_in_geohash_and_blk = data.groupby(['geohash_id', 'block_group_id'])['block_group_id'].count().reset_index(name="total_stays_in_geohash_and_blk")
    merged_df = pd.merge(total_stays_in_geohash, total_stays_in_geohash_and_blk, how="inner", on="geohash_id")
    
    weights = merged_df["total_stays_in_geohash_and_blk"] / merged_df["total_stays_in_geohash"]
    merged_df.insert(loc=4, column='weight', value=weights.values)
    merged_df['weight'] = merged_df['weight'].fillna(1)
    
#     print(merged_df[merged_df['weight'] < 1])

    return merged_df

def crosswalk_tract_geohash(data):
    """
    output a mapping between geohash ids and block group ids. 
    assign a weight to those mappings using the following method:
    (1) find the total number of stays per geohash id 
    (2) find the total number of stays per geohash id and tract 
    the weight is equal to (2) divided by (1)
    """
    
    data = data[['geohash_id', 'tract_id']]
    total_stays_in_geohash = data.groupby('geohash_id')['tract_id'].count().reset_index(name="total_stays_in_geohash")
    total_stays_in_geohash_and_tract = data.groupby(['geohash_id', 'tract_id'])['tract_id'].count().reset_index(name="total_stays_in_geohash_and_tract")
    merged_df = pd.merge(total_stays_in_geohash, total_stays_in_geohash_and_tract, how="inner", on="geohash_id")
    
    weights = merged_df["total_stays_in_geohash_and_tract"] / merged_df["total_stays_in_geohash"]
    merged_df.insert(loc=4, column='weight', value=weights.values)
    merged_df['weight'] = merged_df['weight'].fillna(1)
    
#     print(merged_df[merged_df['weight'] < 1])

    return merged_df

<font color='blue'>**For Example:   
function "13.crosswalk_tract_geohash"**</font>

In [56]:
# to record the time when the program starts executing
exposure_start_time = time.time()

geohash_tract_crosswalk_test = crosswalk_tract_geohash(filtered_stop_data_test)

# to record the time when the program finish executing
print("total executing time", (time.time() - exposure_start_time)/60, "minutes to run")

geohash_tract_crosswalk_test.info()
geohash_tract_crosswalk_test.head()

total executing time 0.03406110604604085 minutes to run
<class 'pandas.core.frame.DataFrame'>
Int64Index: 242008 entries, 0 to 242007
Data columns (total 5 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   geohash_id                        242008 non-null  object 
 1   total_stays_in_geohash            242008 non-null  int64  
 2   tract_id                          242008 non-null  object 
 3   total_stays_in_geohash_and_tract  242008 non-null  int64  
 4   weight                            242008 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 11.1+ MB


,geohash_id,total_stays_in_geohash,tract_id,total_stays_in_geohash_and_tract,weight
0,dp3efryy,1,US.IL.031.17031830008,1,1.0
1,dp3efv7e,1,US.IL.031.17031830008,1,1.0
2,dp3efwpv,1,US.IL.031.17031830008,1,1.0
3,dp3efwpy,1,US.IL.031.17031830008,1,1.0
4,dp3efy00,1,US.IL.031.17031830008,1,1.0


## 14.The Function of `"aggregate_geohash_intervals"`  

**Explanation:**  
This function is to aggregate the <font color='blue'>**<u>"weighted average" racial exposures</u>**</font> into the same tract id by <font color='blue'>**<u>5 time intervals separately</u>**</font> at given date.

In [57]:
def aggregate_geohash(time_aggregate_df, geohash_tract_crosswalk, date):
    '''
    This function aggregates measaures to the tract level, 
    '''
    merged_df = pd.merge(time_aggregate_df, geohash_tract_crosswalk, on="geohash_id")
    
    ## First calculated the exporsure for each piece of geohash
#     merged_df["white_weighted_exposure"] = merged_df['white_exposure'] * merged_df["weight"]
#     merged_df["black_weighted_exposure"] = merged_df['black_exposure'] * merged_df["weight"]
#     merged_df["asian_weighted_exposure"] = merged_df['asian_exposure'] * merged_df["weight"]
#     merged_df["hispanic_weighted_exposure"] = merged_df['hispanic_exposure'] * merged_df["weight"]
#     merged_df["other_weighted_exposure"] = merged_df["other_exposure"] * merged_df["weight"]
    
    merged_df["white_weighted_exposure"] = merged_df['white_exposure']
    merged_df["black_weighted_exposure"] = merged_df['black_exposure']
    merged_df["asian_weighted_exposure"] = merged_df['asian_exposure']
    merged_df["hispanic_weighted_exposure"] = merged_df['hispanic_exposure']
    merged_df["other_weighted_exposure"] = merged_df["other_exposure"]

    merged_df["total_weighted_exposure"] =  merged_df["white_weighted_exposure"] +  merged_df["black_weighted_exposure"]+ merged_df["asian_weighted_exposure"] + merged_df["hispanic_weighted_exposure"] + merged_df["other_weighted_exposure"]
    # Then, for each piece of geohash in the tract, 
    # average the exposure in the tract by the piece and by how many people each 
    # geohash piece represents
    
    # original:    wm = lambda x: np.average(x, weights = merged_df.loc[x.index, "total_stays_in_geohash_and_tract"])
    # df is not a paramter for np.average
    wm = lambda x: np.average(x, weights = merged_df.loc[x.index, "total_stays_in_geohash_and_tract"])

    aggregated_df_w = merged_df[merged_df['white_weighted_exposure']!=0].groupby('tract_id').agg(white_weighted_exposure = ('white_weighted_exposure',wm)).reset_index()
    aggregated_df_b = merged_df[merged_df['black_weighted_exposure']!=0].groupby('tract_id').agg(black_weighted_exposure = ('black_weighted_exposure',wm)).reset_index()
    aggregated_df_h = merged_df[merged_df['hispanic_weighted_exposure']!=0].groupby('tract_id').agg(hispanic_weighted_exposure = ('hispanic_weighted_exposure',wm)).reset_index()
    aggregated_df_a = merged_df[merged_df['asian_weighted_exposure']!=0].groupby('tract_id').agg(asian_weighted_exposure = ('asian_weighted_exposure',wm)).reset_index()
    aggregated_df_o = merged_df[merged_df['other_weighted_exposure']!=0].groupby('tract_id').agg(other_weighted_exposure = ('other_weighted_exposure',wm)).reset_index()
    aggregated_df_t = merged_df[merged_df['total_weighted_exposure']!=0].groupby('tract_id').agg(total_weighted_exposure = ('total_weighted_exposure',wm)).reset_index()
    aggregated_df = aggregated_df_w.merge(aggregated_df_b,on='tract_id').merge(aggregated_df_h,on='tract_id').merge(aggregated_df_a,on='tract_id').merge(aggregated_df_o,on='tract_id').merge(aggregated_df_t,on='tract_id')
        
    aggregated_df["date"] = date.strftime('%Y-%m-%d')
    return aggregated_df[["tract_id", "white_weighted_exposure", "black_weighted_exposure", "asian_weighted_exposure", "hispanic_weighted_exposure","other_weighted_exposure", "total_weighted_exposure", "date"]]

def aggregate_geohash_intervals(time_aggregate_dfs, geohash_tract_crosswalk, date):
    '''
    This function aggregates measaures to the tract level, 
    '''
    agg_geo_int_arr = []
    for label, time_aggregate_df in time_aggregate_dfs.items():
        merged_df = pd.merge(time_aggregate_df, geohash_tract_crosswalk, on="geohash_id")
        ## First calculated the exporsure for each piece of geohash
    #     merged_df["white_weighted_exposure"] = merged_df['white_exposure'] * merged_df["weight"]
    #     merged_df["black_weighted_exposure"] = merged_df['black_exposure'] * merged_df["weight"]
    #     merged_df["asian_weighted_exposure"] = merged_df['asian_exposure'] * merged_df["weight"]
    #     merged_df["hispanic_weighted_exposure"] = merged_df['hispanic_exposure'] * merged_df["weight"]
    #     merged_df["other_weighted_exposure"] = merged_df["other_exposure"] * merged_df["weight"]

        merged_df["white_weighted_exposure"] = merged_df['white_exposure']
        merged_df["black_weighted_exposure"] = merged_df['black_exposure']
        merged_df["asian_weighted_exposure"] = merged_df['asian_exposure']
        merged_df["hispanic_weighted_exposure"] = merged_df['hispanic_exposure']
        merged_df["other_weighted_exposure"] = merged_df["other_exposure"]

        merged_df["total_weighted_exposure"] =  merged_df["white_weighted_exposure"] +  merged_df["black_weighted_exposure"]+ merged_df["asian_weighted_exposure"] + merged_df["hispanic_weighted_exposure"] + merged_df["other_weighted_exposure"]
        # Then, for each piece of geohash in the tract, 
        # average the exposure in the tract by the piece and by how many people each 
        # geohash piece represents

        # original:    wm = lambda x: np.average(x, weights = merged_df.loc[x.index, "total_stays_in_geohash_and_tract"])
        # df is not a paramter for np.average
        wm = lambda x: np.average(x, weights = merged_df.loc[x.index, "total_stays_in_geohash_and_tract"])

        aggregated_df_w = merged_df[merged_df['white_weighted_exposure']!=0].groupby('tract_id').agg(white_weighted_exposure = ('white_weighted_exposure',wm)).reset_index()
        aggregated_df_b = merged_df[merged_df['black_weighted_exposure']!=0].groupby('tract_id').agg(black_weighted_exposure = ('black_weighted_exposure',wm)).reset_index()
        aggregated_df_h = merged_df[merged_df['hispanic_weighted_exposure']!=0].groupby('tract_id').agg(hispanic_weighted_exposure = ('hispanic_weighted_exposure',wm)).reset_index()
        aggregated_df_a = merged_df[merged_df['asian_weighted_exposure']!=0].groupby('tract_id').agg(asian_weighted_exposure = ('asian_weighted_exposure',wm)).reset_index()
        aggregated_df_o = merged_df[merged_df['other_weighted_exposure']!=0].groupby('tract_id').agg(other_weighted_exposure = ('other_weighted_exposure',wm)).reset_index()
        aggregated_df_t = merged_df[merged_df['total_weighted_exposure']!=0].groupby('tract_id').agg(total_weighted_exposure = ('total_weighted_exposure',wm)).reset_index()
        aggregated_df = aggregated_df_w.merge(aggregated_df_b,on='tract_id').merge(aggregated_df_h,on='tract_id').merge(aggregated_df_a,on='tract_id').merge(aggregated_df_o,on='tract_id').merge(aggregated_df_t,on='tract_id')

        aggregated_df["date"] = date.strftime('%Y-%m-%d')
        aggregated_df["interval"] = label
        agg_geo_int_arr.append(aggregated_df[["tract_id", "white_weighted_exposure", "black_weighted_exposure", "asian_weighted_exposure", "hispanic_weighted_exposure","other_weighted_exposure", "total_weighted_exposure", "date", "interval"]])
    return pd.concat(agg_geo_int_arr)

<font color='blue'>**For Example:   
function "14.aggregate_geohash_intervals"**</font>

In [58]:
# to record the time when the program starts executing
exposure_start_time = time.time()

geohash_aggregate_test = aggregate_geohash_intervals(time_aggregate_test, geohash_tract_crosswalk_test, date)

# to record the time when the program finish executing
print("total executing time", (time.time() - exposure_start_time)/60, "minutes to run")

print("Unique time intervals:", geohash_aggregate_test['interval'].unique())
print("Unique dates:", geohash_aggregate_test['date'].unique())

geohash_aggregate_test.info()
geohash_aggregate_test.head()

total executing time 0.22673629522323607 minutes to run
Unique time intervals: ['late night' 'morning' 'afternoon' 'evening' 'late evening']
Unique dates: ['2022-08-01']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5471 entries, 0 to 1072
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tract_id                    5471 non-null   object 
 1   white_weighted_exposure     5471 non-null   float64
 2   black_weighted_exposure     5471 non-null   float64
 3   asian_weighted_exposure     5471 non-null   float64
 4   hispanic_weighted_exposure  5471 non-null   float64
 5   other_weighted_exposure     5471 non-null   float64
 6   total_weighted_exposure     5471 non-null   float64
 7   date                        5471 non-null   object 
 8   interval                    5471 non-null   object 
dtypes: float64(6), object(3)
memory usage: 427.4+ KB


,tract_id,white_weighted_exposure,black_weighted_exposure,asian_weighted_exposure,hispanic_weighted_exposure,other_weighted_exposure,total_weighted_exposure,date,interval
0,US.IL.031.17031010100,0.243093,0.226615,0.020850,0.140987,0.031207,0.662752,2022-08-01,late night
1,US.IL.031.17031010201,0.210382,0.170164,0.114495,0.181754,0.036224,0.713020,2022-08-01,late night
2,US.IL.031.17031010202,0.248949,0.142782,0.055196,0.145886,0.055197,0.648010,2022-08-01,late night
3,US.IL.031.17031010300,0.248869,0.204550,0.034087,0.143651,0.036524,0.667681,2022-08-01,late night
4,US.IL.031.17031010501,0.183620,0.241386,0.040498,0.201451,0.027898,0.694853,2022-08-01,late night


## Final result example test

In [59]:
tract_measures_test = pd.merge(geohash_aggregate_test, tract_sizes_test, how="inner", left_on="tract_id", right_on="home_tract_id").drop(columns = 'home_tract_id')

tract_measures_test.info()
tract_measures_test.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5469 entries, 0 to 5468
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tract_id                    5469 non-null   object 
 1   white_weighted_exposure     5469 non-null   float64
 2   black_weighted_exposure     5469 non-null   float64
 3   asian_weighted_exposure     5469 non-null   float64
 4   hispanic_weighted_exposure  5469 non-null   float64
 5   other_weighted_exposure     5469 non-null   float64
 6   total_weighted_exposure     5469 non-null   float64
 7   date                        5469 non-null   object 
 8   interval                    5469 non-null   object 
 9   cuebiq_id                   5469 non-null   int64  
dtypes: float64(6), int64(1), object(3)
memory usage: 470.0+ KB


,tract_id,white_weighted_exposure,black_weighted_exposure,asian_weighted_exposure,hispanic_weighted_exposure,other_weighted_exposure,total_weighted_exposure,date,interval,cuebiq_id
0,US.IL.031.17031010100,0.243093,0.226615,0.020850,0.140987,0.031207,0.662752,2022-08-01,late night,202
1,US.IL.031.17031010100,0.236812,0.197115,0.024556,0.155208,0.026860,0.640551,2022-08-01,morning,202
2,US.IL.031.17031010100,0.195568,0.218915,0.045680,0.121014,0.026363,0.607540,2022-08-01,afternoon,202
3,US.IL.031.17031010100,0.215448,0.220403,0.041297,0.125560,0.026583,0.629291,2022-08-01,evening,202
4,US.IL.031.17031010100,0.240877,0.235666,0.017592,0.140355,0.027870,0.662360,2022-08-01,late evening,202


In [60]:
filtered_tract_measures_test = tract_measures_test[tract_measures_test['cuebiq_id']>1]
filtered_tract_measures_test['white_weighted_exposure'] = filtered_tract_measures_test['white_weighted_exposure'].astype(str)
filtered_tract_measures_test['black_weighted_exposure'] = filtered_tract_measures_test['black_weighted_exposure'].astype(str)
filtered_tract_measures_test['asian_weighted_exposure'] = filtered_tract_measures_test['asian_weighted_exposure'].astype(str)
filtered_tract_measures_test['hispanic_weighted_exposure'] = filtered_tract_measures_test['hispanic_weighted_exposure'].astype(str)
filtered_tract_measures_test['other_weighted_exposure'] = filtered_tract_measures_test['other_weighted_exposure'].astype(str)
filtered_tract_measures_test['total_weighted_exposure'] = filtered_tract_measures_test['total_weighted_exposure'].astype(str)
filtered_tract_measures_test['cuebiq_id'] = filtered_tract_measures_test['cuebiq_id'].astype(str)
filtered_tract_measures_test = filtered_tract_measures_test.rename(columns={'cuebiq_id':"count_people"})

filtered_tract_measures_test.info()
filtered_tract_measures_test.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5467 entries, 0 to 5468
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   tract_id                    5467 non-null   object
 1   white_weighted_exposure     5467 non-null   object
 2   black_weighted_exposure     5467 non-null   object
 3   asian_weighted_exposure     5467 non-null   object
 4   hispanic_weighted_exposure  5467 non-null   object
 5   other_weighted_exposure     5467 non-null   object
 6   total_weighted_exposure     5467 non-null   object
 7   date                        5467 non-null   object
 8   interval                    5467 non-null   object
 9   count_people                5467 non-null   object
dtypes: object(10)
memory usage: 469.8+ KB


,tract_id,white_weighted_exposure,black_weighted_exposure,asian_weighted_exposure,hispanic_weighted_exposure,other_weighted_exposure,total_weighted_exposure,date,interval,count_people
0,US.IL.031.17031010100,0.24309295211976126,0.22661509069325897,0.020849984725382745,0.1409865628378071,0.031207268220814755,0.6627518585970248,2022-08-01,late night,202
1,US.IL.031.17031010100,0.23681225703309683,0.19711460127676658,0.024556378755608597,0.1552076855957442,0.026860012917966407,0.6405509355791825,2022-08-01,morning,202
2,US.IL.031.17031010100,0.19556758665487567,0.21891538027443805,0.045679744247290655,0.1210139937380512,0.026363254754615997,0.6075399596692715,2022-08-01,afternoon,202
3,US.IL.031.17031010100,0.21544799353636127,0.22040255832613403,0.041297322262767035,0.12555961822608755,0.026583189312293382,0.6292906816636433,2022-08-01,evening,202
4,US.IL.031.17031010100,0.2408765939554614,0.23566603484398124,0.017592006134315854,0.14035503569269667,0.027870070220020287,0.6623597408464753,2022-08-01,late evening,202


## 15.The Function of `"get_exposure_table"`  

**Explanation:**  
This function is to aggregate all the function above into together.

In [61]:
def get_exposure_table(county, date_unformatted):
    time_intervals = [
        ('late night', 0, 6),
        ('morning', 6, 12),
        ('afternoon', 12, 17),
        ('evening', 17, 22),
        ('late evening', 22, 24)
    ]
    exposure_start_time = time.time()
    date = datetime.fromisoformat(date_unformatted)
    stop_data = get_stop_data(county, date)
    stop_data['geohash_id'] =  stop_data['geohash_id'].str[:8]
    if len(stop_data) == 0: return pd.DataFrame()
    filtered_stop_data = remove_nonhuman_devices(stop_data)
    if len(filtered_stop_data) == 0: return pd.DataFrame()
    filtered_stop_data['home_tract_id'] = filtered_stop_data['home_block_group_id'].apply(lambda x: x[:-1])
    filtered_stop_data['tract_id'] = filtered_stop_data['block_group_id'].apply(lambda x: x[:-1])
    filtered_stop_data = filtered_stop_data[~filtered_stop_data['stop_zoned_datetime'].str.contains('Z')]
    
    stop_and_census_data = pd.merge(filtered_stop_data, cw_2010_2020, how="inner", left_on="home_tract_id", right_on="home_tract_id")
    if len(stop_and_census_data) == 0: return pd.DataFrame()
    
    tract_sizes = get_tract_sizes(filtered_stop_data, county)
    
    exposure = get_exposure(stop_and_census_data, date)
    time_aggregate = aggregate_time_intervals(exposure, time_intervals, date)
    
    geohash_tract_crosswalk = crosswalk_tract_geohash(filtered_stop_data)
    geohash_aggregate = aggregate_geohash_intervals(time_aggregate, geohash_tract_crosswalk, date)
    tract_measures = pd.merge(geohash_aggregate, tract_sizes, how="inner", left_on="tract_id", right_on="home_tract_id").drop(columns = 'home_tract_id')
    print("overlap and exposure took", (time.time() - exposure_start_time)/60, "minutes to run")
    ## Changing this to greater than 1 for now. When outputing, will need to aggregate and filter again
    filtered_tract_measures = tract_measures[tract_measures['cuebiq_id']>1]
    filtered_tract_measures['white_weighted_exposure'] = filtered_tract_measures['white_weighted_exposure'].astype(str)
    filtered_tract_measures['black_weighted_exposure'] = filtered_tract_measures['black_weighted_exposure'].astype(str)
    filtered_tract_measures['asian_weighted_exposure'] = filtered_tract_measures['asian_weighted_exposure'].astype(str)
    filtered_tract_measures['hispanic_weighted_exposure'] = filtered_tract_measures['hispanic_weighted_exposure'].astype(str)
    filtered_tract_measures['other_weighted_exposure'] = filtered_tract_measures['other_weighted_exposure'].astype(str)
    filtered_tract_measures['total_weighted_exposure'] = filtered_tract_measures['total_weighted_exposure'].astype(str)
    filtered_tract_measures['cuebiq_id'] = filtered_tract_measures['cuebiq_id'].astype(str)
    filtered_tract_measures = filtered_tract_measures.rename(columns={'cuebiq_id':"count_people"})
    return filtered_tract_measures

## Tracts
def get_exposure_table_many(counties, date_unformatted):
    exposure_start_time = time.time()
    date = datetime.fromisoformat(date_unformatted)
    stop_data = get_stop_data_many(counties, date)
    stop_data['geohash_id'] =  stop_data['geohash_id'].str[:8]
    if len(stop_data) == 0: return pd.DataFrame()
    filtered_stop_data = remove_nonhuman_devices(stop_data)
    if len(filtered_stop_data) == 0: return pd.DataFrame()
    filtered_stop_data['home_tract_id'] = filtered_stop_data['home_block_group_id'].apply(lambda x: x[:-1])
    filtered_stop_data['tract_id'] = filtered_stop_data['block_group_id'].apply(lambda x: x[:-1])
    
    stop_and_census_data = pd.merge(filtered_stop_data, cw_2010_2020, how="inner", left_on="home_tract_id", right_on="home_tract_id")
    if len(stop_and_census_data) == 0: return pd.DataFrame()
    
    tract_sizes = get_tract_sizes_many(filtered_stop_data)
    
    exposure = get_exposure(stop_and_census_data, date)
    time_aggregate = aggregate_time(exposure)
    
    geohash_tract_crosswalk = crosswalk_tract_geohash(filtered_stop_data)
    geohash_aggregate = aggregate_geohash(time_aggregate, geohash_tract_crosswalk, date)
    tract_measures = pd.merge(geohash_aggregate, tract_sizes, how="inner", left_on="tract_id", right_on="home_tract_id").drop(columns = 'home_tract_id')
    print("overlap and exposure took", (time.time() - exposure_start_time)/60, "minutes to run")
    ## Changing this to greater than 1 for now. When outputing, will need to aggregate and filter again
    filtered_tract_measures = tract_measures[tract_measures['cuebiq_id']>1]
    filtered_tract_measures['white_weighted_exposure'] = filtered_tract_measures['white_weighted_exposure'].astype(str)
    filtered_tract_measures['black_weighted_exposure'] = filtered_tract_measures['black_weighted_exposure'].astype(str)
    filtered_tract_measures['asian_weighted_exposure'] = filtered_tract_measures['asian_weighted_exposure'].astype(str)
    filtered_tract_measures['hispanic_weighted_exposure'] = filtered_tract_measures['hispanic_weighted_exposure'].astype(str)
    filtered_tract_measures['other_weighted_exposure'] = filtered_tract_measures['other_weighted_exposure'].astype(str)
    filtered_tract_measures['total_weighted_exposure'] = filtered_tract_measures['total_weighted_exposure'].astype(str)
    filtered_tract_measures['cuebiq_id'] = filtered_tract_measures['cuebiq_id'].astype(str)
    filtered_tract_measures = filtered_tract_measures.rename(columns={'cuebiq_id':"count_people"})
    return filtered_tract_measures

<font color='blue'>**For Example:   
function "15.get_exposure_table"**</font>

In [62]:
# to record the time when the program starts executing
exposure_start_time = time.time()

filtered_tract_measures = get_exposure_table(county, date_unformatted)

# to record the time when the program finish executing
print("total executing time", (time.time() - exposure_start_time)/60, "minutes to run")

filtered_tract_measures.info()
filtered_tract_measures.head()

size of data before filtering:  580401
size of data after filtering:  497203
Total difference between max and min time in minutes:  1439.9833333333333
Total number of buckets:  96
overlap and exposure took 9.721513775984446 minutes to run
total executing time 9.744250611464183 minutes to run
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5467 entries, 0 to 5468
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   tract_id                    5467 non-null   object
 1   white_weighted_exposure     5467 non-null   object
 2   black_weighted_exposure     5467 non-null   object
 3   asian_weighted_exposure     5467 non-null   object
 4   hispanic_weighted_exposure  5467 non-null   object
 5   other_weighted_exposure     5467 non-null   object
 6   total_weighted_exposure     5467 non-null   object
 7   date                        5467 non-null   object
 8   interval                    5467 no

,tract_id,white_weighted_exposure,black_weighted_exposure,asian_weighted_exposure,hispanic_weighted_exposure,other_weighted_exposure,total_weighted_exposure,date,interval,count_people
0,US.IL.031.17031010100,0.24309295211976126,0.22661509069325897,0.020849984725382745,0.1409865628378071,0.031207268220814755,0.6627518585970248,2022-08-01,late night,202
1,US.IL.031.17031010100,0.23681225703309683,0.19711460127676658,0.024556378755608597,0.1552076855957442,0.026860012917966407,0.6405509355791825,2022-08-01,morning,202
2,US.IL.031.17031010100,0.19556758665487567,0.21891538027443805,0.045679744247290655,0.1210139937380512,0.026363254754615997,0.6075399596692715,2022-08-01,afternoon,202
3,US.IL.031.17031010100,0.21544799353636127,0.22040255832613403,0.041297322262767035,0.12555961822608755,0.026583189312293382,0.6292906816636433,2022-08-01,evening,202
4,US.IL.031.17031010100,0.2408765939554614,0.23566603484398124,0.017592006134315854,0.14035503569269667,0.027870070220020287,0.6623597408464753,2022-08-01,late evening,202


# Unread function

In [ ]:
def curr_county_and_date (counties, dates):
    current_info_arr = sql_engine.read_sql(f"""
         select *
         from dedicated.incrementors.current_urban
         """)
    current_info_arr['county']=current_info_arr['county'].astype(int)
    current_info_arr['date']=current_info_arr['date'].astype(int)
    current_info_arr= current_info_arr.to_records()
    current_info_arr.sort(order=['date', 'county'])
    current_info = current_info_arr[-1]
    county_idx = int(current_info[1])
    date_idx = int(current_info[2])
    
    counties_len = len(counties)
    dates_len = len(dates)
    
    if county_idx>=counties_len: return (county_idx, date_idx, counties_len, dates_len, '', '')
    county_ref = counties[county_idx]
    if date_idx>=dates_len: return     (county_idx, date_idx, counties_len, dates_len, '', '')
    date_ref = dates[date_idx]
    return (county_idx, date_idx, counties_len, dates_len, county_ref, date_ref)

def curr_county_and_date_many (counties, dates, count):
    current_info_arr = sql_engine.read_sql(f"""
         select *
         from dedicated.incrementors.current_rural
         """)
    current_info_arr['county']=current_info_arr['county'].astype(int)
    current_info_arr['date']=current_info_arr['date'].astype(int)
    current_info_arr= current_info_arr.to_records()
    current_info_arr.sort(order=['date', 'county'])
    current_info = current_info_arr[-1]
    county_idx = int(current_info[1])
    date_idx = int(current_info[2])
    
    counties_len = len(counties)
    dates_len = len(dates)
    
    if county_idx>=counties_len: return (county_idx, date_idx, counties_len, dates_len, '', '')
    county_ref_arr = counties[county_idx:county_idx+count]
    if date_idx>=dates_len: return (county_idx, date_idx, counties_len, dates_len, '', '')
    date_ref = dates[date_idx]
    return (county_idx, date_idx, counties_len, dates_len, county_ref_arr, date_ref)

## Altering Date Range

In [53]:
# Sept 12 to Sept 18 (inclusive), 2022, Dec 12 to Dec 19 (inclusive) 2022, March 6 to 12 (inclusive), 2023

# %sql delete from dedicated.incrementors.dates
# insert_into_db_inc('dates', pd.DataFrame({'date':['2022-06-06', '2022-06-07', '2022-06-08', '2022-06-09', '2022-06-10', '2022-06-11', '2022-06-12']}),10)
# insert_into_db_inc('dates', pd.DataFrame({'date':['2022-09-12', '2022-09-13', '2022-09-14', '2022-09-15', '2022-09-16', '2022-09-17', '2022-09-18']}),10)
# insert_into_db_inc('dates', pd.DataFrame({'date':['2022-12-12', '2022-12-13', '2022-12-14', '2022-12-15', '2022-12-16', '2022-12-17', '2022-12-18']}),10)
# insert_into_db_inc('dates', pd.DataFrame({'date':['2022-03-07', '2022-03-08', '2022-03-09', '2022-03-10', '2022-03-11', '2022-03-12', '2022-03-13']}),10)

## Resetting data

### Restart incrementors

In [54]:
# # to restart incrementor for rural
# %sql delete from dedicated.incrementors.current_rural
# insert_into_db_inc('current_rural', pd.DataFrame({'county': ['0'], 'date': ['0']}),5)

# # to restart incrementor for populous
# %sql delete from dedicated.incrementors.current_urban
# insert_into_db_inc('current_urban', pd.DataFrame({'county':['0'], 'date':['0']}),5)

### Delete current data

In [88]:
# %sql delete from dedicated.exposure_scaled.county_rural_prod
# %sql delete from dedicated.incrementors.failed_counties_rural

# %sql delete from dedicated.exposure_scaled.county_urban_prod
# %sql delete from dedicated.incrementors.failed_counties_urban

## Exposure Calculations

In [89]:
# m_exposure_start_time = time.time()
# counties = sql_engine.read_sql(f"""
#              select *
#              from dedicated.exposure_scaled.county_1
#              """)['county'].values
# counties.sort()
# dates = sql_engine.read_sql(f"""
#              select *
#              from dedicated.incrementors.dates
#              """)['date'].values
# dates.sort()

# counties_len = len(counties)
# dates_len = len(dates)

In [90]:
# current_info = sql_engine.read_sql(f"""
#          select *
#          from dedicated.incrementors.current_urban
#          """)['current'].values[0].split(',')
# county_idx = int(current_info[0])
# date_idx = int(current_info[1])
# county_ref = counties[county_idx]
# date_ref = dates[date_idx]

In [91]:
# county_idx, date_idx, counties_len, dates_len, county_ref, date_ref = curr_county_and_date(counties, dates)

In [92]:
# current_info

In [93]:
# test2[test2.date=='2022-06-09']['county'].unique()

In [94]:
# insert_into_db_inc('current_urban', pd.DataFrame({'current':[str(15+1)+','+str(3)]}),5)

In [95]:
# current_info

In [97]:
# current_info = sql_engine.read_sql(f"""
#      select *
#      from dedicated.incrementors.current_urban
#      """)['current'].values[0].split(',')
# county_idx = int(current_info[0])
# date_idx = int(current_info[1])
# county_ref = counties[county_idx]
# date_ref = dates[date_idx]

In [98]:
# county_exposure = get_exposure_table(county_ref, date_ref)

In [99]:
# insert_into_db("county_urban_prod", county_exposure, 5000)

In [100]:
# %sql delete from dedicated.incrementors.current_urban

In [101]:
# insert_into_db_inc('current_urban', pd.DataFrame({'current':[str(county_idx+1)+','+str(date_idx)]}),5)

In [102]:
# sql_engine.read_sql('select * from dedicated.incrementors.current_urban')

In [192]:
# FEW
m_exposure_start_time = time.time()
counties = sql_engine.read_sql(f"""
             select *
             from dedicated.exposure_scaled.county_1
             """)['county'].values
counties.sort()
dates = sql_engine.read_sql(f"""
             select *
             from dedicated.incrementors.dates
             """)['date'].values
dates.sort()

counties_len = len(counties)
dates_len = len(dates)

while True:
    current_info_arr = sql_engine.read_sql(f"""
         select *
         from dedicated.incrementors.current_urban
         """)
    current_info_arr['county']=current_info_arr['county'].astype(int)
    current_info_arr['date']=current_info_arr['date'].astype(int)
    current_info_arr= current_info_arr.to_records()
    current_info_arr.sort(order=['date', 'county'])
    current_info = current_info_arr[-1]
    county_idx = int(current_info[1])
    date_idx = int(current_info[2])
    if date_idx>=dates_len: break
    if county_idx>=counties_len: break
    county_ref = counties[county_idx]
    date_ref = dates[date_idx]
    while county_idx < counties_len:
        try:
            print('county: ' + county_ref)
            print('date: ' + date_ref)
            county_exposure = get_exposure_table(county_ref, date_ref)
            insert_into_db("county_urban_prod_3", county_exposure, 5000)
            # delete current indicator from table
#             %sql delete from dedicated.incrementors.current_urban
        except:
            insert_into_db_inc('failed_counties_urban', pd.DataFrame({'county': county_ref, 'date': date_ref}), 5)
#             %sql delete from dedicated.incrementors.current_urban
        insert_into_db_inc('current_urban', pd.DataFrame({'county':[str(county_idx+1)], 'date':[str(date_idx)]}),5)
        county_idx, date_idx, counties_len, dates_len, county_ref, date_ref = curr_county_and_date(counties, dates)
#     %sql delete from dedicated.incrementors.current_urban
    insert_into_db_inc('current_urban', pd.DataFrame({'county':['0'], 'date':[str(date_idx+1)]}),5)
print("overlap and exposure took", (time.time() - m_exposure_start_time)/60, "minutes to run")


KeyboardInterrupt: 

In [86]:
counties = sql_engine.read_sql(f"""
             select *
             from dedicated.exposure_scaled.county_1
             """)['county'].values
counties.sort()

In [87]:
counties

array(['US.AZ.013', 'US.AZ.019', 'US.CA.001', 'US.CA.013', 'US.CA.019',
       'US.CA.029', 'US.CA.037', 'US.CA.059', 'US.CA.065', 'US.CA.067',
       'US.CA.071', 'US.CA.073', 'US.CA.075', 'US.CA.077', 'US.CA.081',
       'US.CA.085', 'US.CA.111', 'US.CO.031', 'US.CO.041', 'US.CT.001',
       'US.CT.003', 'US.CT.009', 'US.FL.011', 'US.FL.031', 'US.FL.057',
       'US.FL.071', 'US.FL.086', 'US.FL.095', 'US.FL.099', 'US.FL.103',
       'US.FL.105', 'US.GA.067', 'US.GA.089', 'US.GA.121', 'US.GA.135',
       'US.HI.003', 'US.IL.031', 'US.IL.043', 'US.IL.097', 'US.IL.197',
       'US.IN.097', 'US.KY.111', 'US.MA.009', 'US.MA.017', 'US.MA.021',
       'US.MA.025', 'US.MA.027', 'US.MD.005', 'US.MD.031', 'US.MD.033',
       'US.MI.099', 'US.MI.125', 'US.MI.163', 'US.MN.053', 'US.MO.095',
       'US.MO.189', 'US.MO.510', 'US.NC.119', 'US.NC.183', 'US.NJ.003',
       'US.NJ.013', 'US.NJ.017', 'US.NJ.023', 'US.NM.001', 'US.NV.003',
       'US.NY.005', 'US.NY.029', 'US.NY.047', 'US.NY.055', 'US.N

In [88]:
dates = sql_engine.read_sql(f"""
             select *
             from dedicated.incrementors.dates
             """)['date'].values
dates.sort()

In [89]:
dates

array(['2022-03-07', '2022-03-08', '2022-03-09', '2022-03-10',
       '2022-03-11', '2022-03-12', '2022-03-13', '2022-06-06',
       '2022-06-07', '2022-06-08', '2022-06-09', '2022-06-10',
       '2022-06-11', '2022-06-12', '2022-09-12', '2022-09-13',
       '2022-09-14', '2022-09-15', '2022-09-16', '2022-09-17',
       '2022-09-18', '2022-12-12', '2022-12-13', '2022-12-14',
       '2022-12-15', '2022-12-16', '2022-12-17', '2022-12-18'],
      dtype=object)

In [90]:
counties_len = len(counties)
dates_len = len(dates)
print("counties_len:",counties_len)
print("dates_len:",len(dates))

counties_len: 101
dates_len: 28


In [91]:
current_info_arr = sql_engine.read_sql(f"""
         select *
         from dedicated.incrementors.current_urban
         """)
current_info_arr

,county,date
0,37,0
1,61,0
2,12,0
3,34,0
4,0,0
...,...,...
2000,33,19
2001,39,19
2002,44,19
2003,54,19


In [92]:
current_info_arr['county']=current_info_arr['county'].astype(int)
current_info_arr['date']=current_info_arr['date'].astype(int)
current_info_arr= current_info_arr.to_records()
current_info_arr

rec.array([(   0, 37,  0), (   1, 61,  0), (   2, 12,  0), ...,
           (2002, 44, 19), (2003, 54, 19), (2004, 40, 19)],
          dtype=[('index', '<i8'), ('county', '<i8'), ('date', '<i8')])

In [93]:
current_info_arr.sort(order=['date', 'county'])
current_info_arr

rec.array([(   4,  0,  0), (1033,  1,  0), (1014,  2,  0), ...,
           (1999, 64, 19), (1487, 65, 19), (1482, 66, 19)],
          dtype=[('index', '<i8'), ('county', '<i8'), ('date', '<i8')])

In [94]:
current_info = current_info_arr[-1]
current_info

(1482, 66, 19)

In [95]:
county_idx = int(current_info[1])
date_idx = int(current_info[2])
print(county_idx)
print(date_idx)

66
19


In [96]:
county_ref = counties[county_idx]
date_ref = dates[date_idx]

print(county_ref)
print(date_ref)

US.NY.029
2022-09-17


In [ ]:
# current_info_arr = sql_engine.read_sql(f"""
#          select *
#          from dedicated.incrementors.current_rural
#          """).to_records()
# current_info_arr.sort(order=['date', 'county'])
# current_info_arr
# # current_info = current_info_arr[-1]
# # county_idx = int(current_info[1])
# # date_idx = int(current_info[2])

In [155]:
# exposure_start_time = time.time()
# date = datetime.fromisoformat(date_ref)
# stop_data = get_stop_data_many(county_ref_arr, date)
# stop_data['geohash_id'] =  stop_data['geohash_id'].str[:8]

# filtered_stop_data = remove_nonhuman_devices(stop_data)
# filtered_stop_data['home_tract_id'] = filtered_stop_data['home_block_group_id'].apply(lambda x: x[:-1])
# filtered_stop_data['tract_id'] = filtered_stop_data['block_group_id'].apply(lambda x: x[:-1])

# stop_and_census_data = pd.merge(filtered_stop_data, cw_2010_2020, how="inner", left_on="home_tract_id", right_on="home_tract_id")

# tract_sizes = get_tract_sizes_many(filtered_stop_data)

# exposure = get_exposure(stop_and_census_data, date)
# time_aggregate = aggregate_time(exposure)

# geohash_tract_crosswalk = crosswalk_tract_geohash(filtered_stop_data)
# geohash_aggregate = aggregate_geohash(time_aggregate, geohash_tract_crosswalk, date)
# tract_measures = pd.merge(geohash_aggregate, tract_sizes, how="inner", left_on="tract_id", right_on="home_tract_id").drop(columns = 'home_tract_id')
# print("overlap and exposure took", (time.time() - exposure_start_time)/60, "minutes to run")
# ## Changing this to greater than 1 for now. When outputing, will need to aggregate and filter again
# filtered_tract_measures = tract_measures[tract_measures['cuebiq_id']>1]
# filtered_tract_measures['white_weighted_exposure'] = filtered_tract_measures['white_weighted_exposure'].astype(str)
# filtered_tract_measures['black_weighted_exposure'] = filtered_tract_measures['black_weighted_exposure'].astype(str)
# filtered_tract_measures['asian_weighted_exposure'] = filtered_tract_measures['asian_weighted_exposure'].astype(str)
# filtered_tract_measures['hispanic_weighted_exposure'] = filtered_tract_measures['hispanic_weighted_exposure'].astype(str)
# filtered_tract_measures['other_weighted_exposure'] = filtered_tract_measures['other_weighted_exposure'].astype(str)
# filtered_tract_measures['total_weighted_exposure'] = filtered_tract_measures['total_weighted_exposure'].astype(str)
# filtered_tract_measures['cuebiq_id'] = filtered_tract_measures['cuebiq_id'].astype(str)
# filtered_tract_measures = filtered_tract_measures.rename(columns={'cuebiq_id':"count_people"})

In [ ]:
# counties = sql_engine.read_sql(f"""
#              select *
#              from dedicated.exposure_scaled.county_1
#              """)['county'].values
# counties.sort()
# dates = sql_engine.read_sql(f"""
#              select *
#              from dedicated.incrementors.dates
#              """)['date'].values
# dates.sort()
# dates = ['2022-08-01','2022-08-02']

In [ ]:
# count=10
# county_idx, date_idx, counties_len, dates_len, county_ref, date_ref = curr_county_and_date(counties, dates)
# dates_len = 2
# while date_idx < dates_len:
#     while county_idx + count < counties_len :
#         try:
#             print('county: '+str(counties[county_idx:county_idx+count])+', date: '+ str(dates[date_idx]))
#             county_idx+=count
#         except:
#             county_idx+=count
#     county_idx = 0
#     date_idx+=1

## Read failed counties

In [71]:
# failed_rural = sql_engine.read_sql(f"""
#            select *
#            from dedicated.incrementors.failed_counties_rural
#            """)
# failed_rural
# failed_urban = sql_engine.sql_engine.read_sql(f"""
#            select *
#            from dedicated.incrementors.failed_counties_urban
#            """)
# failed_urban

## Rerun failed counties

In [73]:
# failed = sql_engine.sql_engine.read_sql(f"""
#            select *
#            from dedicated.incrementors.failed_counties_rural
#            """).drop_duplicates()
# counties_5 = sql_engine.sql_engine.read_sql(f"""
#            select *
#            from dedicated.exposure_scaled.county_5
#            """)
# failed = failed.merge(counties_5, how='inner', left_on='county', right_on='county')
# failed.values
# for item in failed.values:
#     print('county: '+ item[0])
#     print('date: '+ item[1])
#     get_exposure_table(item[0], item[1])